In [ ]:
import os
from coffea import hist
from coffea import util
import numpy as np
import itertools

# Start by loading the unweighted coffea outputs containing the efficiency numerator and denominators

In [ ]:
dir1 = 'CoffeaOutputs/UnweightedOutputs/'

QCD_unweighted = util.load(dir1+'TTbarResCoffea_QCD_unweighted_output.coffea')
TTbar_unweighted = util.load(dir1+'TTbarResCoffea_TTbar_unweighted_output.coffea')
RSGluon1000_unweighted = util.load(dir1+'TTbarResCoffea_RSGluon1000_unweighted_output.coffea')
# TestSample_unweighted = util.load(dir1+'TTbarResCoffea_TestSample_ttbarRES_unweighted_output_JohanTest.coffea')
TTbar_biased_1000toInf_unweighted = util.load(dir1+'TTbarResCoffea_TTbar_biased_1000toInf_unweighted_output.coffea')
TTbar_biased_700to1000_unweighted = util.load(dir1+'TTbarResCoffea_TTbar_biased_700to1000_unweighted_output.coffea')

In [ ]:
Datasets = {
    'QCD': QCD_unweighted,
    'TTbar': TTbar_unweighted,
    'RSGluon1000': RSGluon1000_unweighted,
    'TTbar_biased_1000toInf': TTbar_biased_1000toInf_unweighted,
    'TTbar_biased_700to1000': TTbar_biased_700to1000_unweighted
}

# The cutflow can be checked if desired 

In [ ]:
for dataset,output in Datasets.items():
    print("-------" + dataset + " Cutflow--------")
    for i,j in output['cutflow'].items():        
        print( '%20s : %12d' % (i,j) )

# Define a function that can make directories to store png files of the histograms (if desired)

In [ ]:
def mkdir_p(mypath):
    '''Creates a directory. equivalent to using mkdir -p on the command line'''

    from errno import EEXIST
    from os import makedirs,path

    try:
        makedirs(mypath)
    except OSError as exc: # Python >2.5
        if exc.errno == EEXIST and path.isdir(mypath):
            pass
        else: raise

In [ ]:
def DoesDirectoryExist(mypath): #extra precaution (Probably overkill...)
    '''Checks to see if Directory exists before running mkdir_p'''
    import os.path
    from os import path
    
    if path.exists(mypath):
        pass
    else:
        mkdir_p(mypath)

# Define where you want the main directory (place to store the directories with the new png plots)

In [ ]:
maindirectory = os.getcwd() 

Various plotting options I've copied and pasted over from the other analysis notebooks.  Could be of use ;)

In [ ]:
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import warnings
import re # regular expressions
warnings.filterwarnings("ignore")

# stack_tag1_opts = {'marker': '.', 'markersize': 10., 'color': 'b', 'elinewidth': 1}
# stack_tag2_opts = {'marker': 's', 'markersize': 5., 'color': 'g', 'elinewidth': 1}
# legend_tag_opts = {'labels':['', '', 'DeepTag', 'DeepTag MD']}
# stack_ttbar_opts = {'alpha': 0.8, 'edgecolor':(0,0,0,0.3), 'color': 'red'}
# stack_background_opts = {'alpha': 0.8, 'edgecolor':(0,0,0,0.3), 'color': 'yellow'}
# stack_error_opts = {'label':'Stat. Unc.', 'hatch':'///', 'facecolor':'None', 'edgecolor':(0,0,0,.5), 'linewidth': 0}
# data_err_opts = {'linestyle': 'none', 'marker': '.', 'markersize': 10., 'color': 'k', 'elinewidth': 1}

In [ ]:
PlotWithLargeBins = True

# Plot (and save) the b quark tagging efficiencies as functions of $p_T$

In [ ]:
SaveDirectory = maindirectory + '/FlavorTagEfficiencies/images/1D/btags_pt/' # split histograms into subdirectories
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
if PlotWithLargeBins == True:
    binwidth = '_manualbins'
    filename_bw = '_large_bins'
else:
    binwidth = ''
    filename_bw = ''

plt.rcParams.update({
'font.size': 14,
'axes.titlesize': 18,
'axes.labelsize': 18,
'xtick.labelsize': 12,
'ytick.labelsize': 12
})
fig, (ax01, ax02, ax11, ax12) = plt.subplots(1, 4, figsize=(17,5), sharex=True)

b_eff_numerator_pt_s01_biased1 = TTbar_biased_700to1000_unweighted['b_eff_numerator_s01'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
b_eff_numerator_pt_s01_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_numerator_s01'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')
b_eff_denominator_pt_s01_biased1 = TTbar_biased_700to1000_unweighted['b_eff_denominator_s01'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
b_eff_denominator_pt_s01_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_denominator_s01'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')

b_eff_numerator_pt_s01 = b_eff_numerator_pt_s01_biased1.copy()
b_eff_numerator_pt_s01.add(b_eff_numerator_pt_s01_biased2)
b_eff_denominator_pt_s01 = b_eff_denominator_pt_s01_biased1.copy()
b_eff_denominator_pt_s01.add(b_eff_denominator_pt_s01_biased2)

b_eff_pt_s01 = hist.plotratio(num = b_eff_numerator_pt_s01, denom = b_eff_denominator_pt_s01, 
                              ax=ax01, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                              unc = 'num')


ax01.set_title(r'SubJet$_{01}\left(p_T\right)$')
ax01.set_ylabel('b-Tag Efficiency')
ax01.set_ylim(bottom=0., top=1.)

# ------------------------------------------------------------------------------ #

b_eff_numerator_pt_s02_biased1 = TTbar_biased_700to1000_unweighted['b_eff_numerator_s02'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
b_eff_numerator_pt_s02_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_numerator_s02'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')
b_eff_denominator_pt_s02_biased1 = TTbar_biased_700to1000_unweighted['b_eff_denominator_s02'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
b_eff_denominator_pt_s02_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_denominator_s02'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')

b_eff_numerator_pt_s02 = b_eff_numerator_pt_s02_biased1.copy()
b_eff_numerator_pt_s02.add(b_eff_numerator_pt_s02_biased2)
b_eff_denominator_pt_s02 = b_eff_denominator_pt_s02_biased1.copy()
b_eff_denominator_pt_s02.add(b_eff_denominator_pt_s02_biased2)

b_eff_pt_s02 = hist.plotratio(num = b_eff_numerator_pt_s02, denom = b_eff_denominator_pt_s02, 
                              ax=ax02, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                              unc = 'num')


ax02.set_title(r'SubJet$_{02}\left(p_T\right)$')
ax02.set_ylabel('')
ax02.set_ylim(bottom=0., top=1.)

# # ------------------------------------------------------------------------------ #

b_eff_numerator_pt_s11_biased1 = TTbar_biased_700to1000_unweighted['b_eff_numerator_s11'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
b_eff_numerator_pt_s11_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_numerator_s11'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')
b_eff_denominator_pt_s11_biased1 = TTbar_biased_700to1000_unweighted['b_eff_denominator_s11'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
b_eff_denominator_pt_s11_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_denominator_s11'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')

b_eff_numerator_pt_s11 = b_eff_numerator_pt_s11_biased1.copy()
b_eff_numerator_pt_s11.add(b_eff_numerator_pt_s11_biased2)
b_eff_denominator_pt_s11 = b_eff_denominator_pt_s11_biased1.copy()
b_eff_denominator_pt_s11.add(b_eff_denominator_pt_s11_biased2)

b_eff_pt_s11 = hist.plotratio(num = b_eff_numerator_pt_s11, denom = b_eff_denominator_pt_s11, 
                              ax=ax11, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                              unc = 'num')


ax11.set_title(r'SubJet$_{11}\left(p_T\right)$')
ax11.set_ylabel('')
ax11.set_ylim(bottom=0., top=1.)

# # ------------------------------------------------------------------------------ #

b_eff_numerator_pt_s12_biased1 = TTbar_biased_700to1000_unweighted['b_eff_numerator_s12'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
b_eff_numerator_pt_s12_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_numerator_s12'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')
b_eff_denominator_pt_s12_biased1 = TTbar_biased_700to1000_unweighted['b_eff_denominator_s12'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
b_eff_denominator_pt_s12_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_denominator_s12'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')

b_eff_numerator_pt_s12 = b_eff_numerator_pt_s12_biased1.copy()
b_eff_numerator_pt_s12.add(b_eff_numerator_pt_s12_biased2)
b_eff_denominator_pt_s12 = b_eff_denominator_pt_s12_biased1.copy()
b_eff_denominator_pt_s12.add(b_eff_denominator_pt_s12_biased2)

b_eff_pt_s12 = hist.plotratio(num = b_eff_numerator_pt_s12, denom = b_eff_denominator_pt_s12, 
                              ax=ax12, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                              unc = 'num')


ax12.set_title(r'SubJet$_{12}\left(p_T\right)$')
ax12.set_ylabel('')
ax12.set_ylim(bottom=0., top=1.)
leg = ax11.legend(labels=[r'$b_{eff}$'], bbox_to_anchor=(0.7, 1.4), loc='upper right')

Dataset = 'TTbarBiasedSum'
fig.suptitle(Dataset + ' Sample')

In [ ]:
SaveDirectory = maindirectory + '/FlavorTagEfficiencies/images/1D/btags_pt/' # split histograms into subdirectories
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
if PlotWithLargeBins == True:
    binwidth = '_manualbins'
    filename_bw = '_large_bins'
else:
    binwidth = ''
    filename_bw = ''
for dataset,output in Datasets.items(): 
    plt.rcParams.update({
    'font.size': 14,
    'axes.titlesize': 18,
    'axes.labelsize': 18,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12
    })
    fig, (ax01, ax02, ax11, ax12) = plt.subplots(1, 4, figsize=(17,5), sharex=True)

    b_eff_numerator_pt_s01 = output['b_eff_numerator_s01'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    b_eff_denominator_pt_s01 = output['b_eff_denominator_s01'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    
    
    b_eff_pt_s01 = hist.plotratio(num = b_eff_numerator_pt_s01, denom = b_eff_denominator_pt_s01, 
                                  ax=ax01, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                                  unc = 'num')

    
    ax01.set_title(r'SubJet$_{01}\left(p_T\right)$')
    ax01.set_ylabel('b-Tag Efficiency')
    ax01.set_ylim(bottom=0., top=1.)

    # ------------------------------------------------------------------------------ #

    b_eff_numerator_pt_s02 = output['b_eff_numerator_s02'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    b_eff_denominator_pt_s02 = output['b_eff_denominator_s02'+binwidth].integrate('subjeteta').integrate('dataset', dataset)

    
    b_eff_pt_s02 = hist.plotratio(num = b_eff_numerator_pt_s02, denom = b_eff_denominator_pt_s02, 
                                  ax=ax02, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                                  unc = 'num')
    
    
    ax02.set_title(r'SubJet$_{02}\left(p_T\right)$')
    ax02.set_ylabel('')
    ax02.set_ylim(bottom=0., top=1.)

    # ------------------------------------------------------------------------------ #

    b_eff_numerator_pt_s11 = output['b_eff_numerator_s11'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    b_eff_denominator_pt_s11 = output['b_eff_denominator_s11'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    
    
    b_eff_pt_s11 = hist.plotratio(num = b_eff_numerator_pt_s11, denom = b_eff_denominator_pt_s11, 
                                  ax=ax11, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                                  unc = 'num')
    
    
    ax11.set_title(r'SubJet$_{11}\left(p_T\right)$')
    ax11.set_ylabel('')
    ax11.set_ylim(bottom=0., top=1.)

    # ------------------------------------------------------------------------------ #

    b_eff_numerator_pt_s12 = output['b_eff_numerator_s12'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    b_eff_denominator_pt_s12 = output['b_eff_denominator_s12'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    
    
    b_eff_pt_s12 = hist.plotratio(num = b_eff_numerator_pt_s12, denom = b_eff_denominator_pt_s12, 
                                  ax=ax12, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                                  unc = 'num')
    
    
    ax12.set_title(r'SubJet$_{12}\left(p_T\right)$')
    ax12.set_ylabel('')
    ax12.set_ylim(bottom=0., top=1.)
    leg = ax11.legend(labels=[r'$b_{eff}$'], bbox_to_anchor=(0.7, 1.4), loc='upper right')
    fig.suptitle(dataset + ' Sample')

    filename = dataset + '_b_efficiency_pt' + filename_bw + '.png'
    print(filename + ' generated')
    plt.savefig(SaveDirectory+filename, bbox_inches="tight")

# Plot (and save) the b quark tagging efficiencies as functions of $\eta$

In [ ]:
SaveDirectory = maindirectory + '/FlavorTagEfficiencies/images/1D/btags_eta/' # split histograms into subdirectories
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
if PlotWithLargeBins == True:
    binwidth = '_manualbins'
    filename_bw = '_large_bins'
else:
    binwidth = ''
    filename_bw = ''

plt.rcParams.update({
'font.size': 14,
'axes.titlesize': 18,
'axes.labelsize': 18,
'xtick.labelsize': 12,
'ytick.labelsize': 12
})
fig, (ax01, ax02, ax11, ax12) = plt.subplots(1, 4, figsize=(17,5), sharex=True)

b_eff_numerator_eta_s01_biased1 = TTbar_biased_700to1000_unweighted['b_eff_numerator_s01'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
b_eff_numerator_eta_s01_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_numerator_s01'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')
b_eff_denominator_eta_s01_biased1 = TTbar_biased_700to1000_unweighted['b_eff_denominator_s01'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
b_eff_denominator_eta_s01_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_denominator_s01'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')

b_eff_numerator_eta_s01 = b_eff_numerator_eta_s01_biased1.copy()
b_eff_numerator_eta_s01.add(b_eff_numerator_eta_s01_biased2)
b_eff_denominator_eta_s01 = b_eff_denominator_eta_s01_biased1.copy()
b_eff_denominator_eta_s01.add(b_eff_denominator_eta_s01_biased2)

b_eff_eta_s01 = hist.plotratio(num = b_eff_numerator_eta_s01, denom = b_eff_denominator_eta_s01, 
                              ax=ax01, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                              unc = 'num')


ax01.set_title(r'SubJet$_{01}\left(\eta\right)$')
ax01.set_ylabel('b-Tag Efficiency')
ax01.set_ylim(bottom=0., top=1.)

# ------------------------------------------------------------------------------ #

b_eff_numerator_eta_s02_biased1 = TTbar_biased_700to1000_unweighted['b_eff_numerator_s02'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
b_eff_numerator_eta_s02_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_numerator_s02'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')
b_eff_denominator_eta_s02_biased1 = TTbar_biased_700to1000_unweighted['b_eff_denominator_s02'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
b_eff_denominator_eta_s02_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_denominator_s02'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')

b_eff_numerator_eta_s02 = b_eff_numerator_eta_s02_biased1.copy()
b_eff_numerator_eta_s02.add(b_eff_numerator_eta_s02_biased2)
b_eff_denominator_eta_s02 = b_eff_denominator_eta_s02_biased1.copy()
b_eff_denominator_eta_s02.add(b_eff_denominator_eta_s02_biased2)

b_eff_eta_s02 = hist.plotratio(num = b_eff_numerator_eta_s02, denom = b_eff_denominator_eta_s02, 
                              ax=ax02, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                              unc = 'num')


ax02.set_title(r'SubJet$_{02}\left(\eta\right)$')
ax02.set_ylabel('')
ax02.set_ylim(bottom=0., top=1.)

# # ------------------------------------------------------------------------------ #

b_eff_numerator_eta_s11_biased1 = TTbar_biased_700to1000_unweighted['b_eff_numerator_s11'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
b_eff_numerator_eta_s11_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_numerator_s11'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')
b_eff_denominator_eta_s11_biased1 = TTbar_biased_700to1000_unweighted['b_eff_denominator_s11'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
b_eff_denominator_eta_s11_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_denominator_s11'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')

b_eff_numerator_eta_s11 = b_eff_numerator_eta_s11_biased1.copy()
b_eff_numerator_eta_s11.add(b_eff_numerator_eta_s11_biased2)
b_eff_denominator_eta_s11 = b_eff_denominator_eta_s11_biased1.copy()
b_eff_denominator_eta_s11.add(b_eff_denominator_eta_s11_biased2)

b_eff_eta_s11 = hist.plotratio(num = b_eff_numerator_eta_s11, denom = b_eff_denominator_eta_s11, 
                              ax=ax11, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                              unc = 'num')


ax11.set_title(r'SubJet$_{11}\left(\eta\right)$')
ax11.set_ylabel('')
ax11.set_ylim(bottom=0., top=1.)

# # ------------------------------------------------------------------------------ #

b_eff_numerator_eta_s12_biased1 = TTbar_biased_700to1000_unweighted['b_eff_numerator_s12'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
b_eff_numerator_eta_s12_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_numerator_s12'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')
b_eff_denominator_eta_s12_biased1 = TTbar_biased_700to1000_unweighted['b_eff_denominator_s12'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
b_eff_denominator_eta_s12_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_denominator_s12'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')

b_eff_numerator_eta_s12 = b_eff_numerator_eta_s12_biased1.copy()
b_eff_numerator_eta_s12.add(b_eff_numerator_eta_s12_biased2)
b_eff_denominator_eta_s12 = b_eff_denominator_eta_s12_biased1.copy()
b_eff_denominator_eta_s12.add(b_eff_denominator_eta_s12_biased2)

b_eff_eta_s12 = hist.plotratio(num = b_eff_numerator_eta_s12, denom = b_eff_denominator_eta_s12, 
                              ax=ax12, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                              unc = 'num')


ax12.set_title(r'SubJet$_{12}\left(\eta\right)$')
ax12.set_ylabel('')
ax12.set_ylim(bottom=0., top=1.)
leg = ax11.legend(labels=[r'$b_{eff}$'], bbox_to_anchor=(0.7, 1.4), loc='upper right')

Dataset = 'TTbarBiasedSum'
fig.suptitle(Dataset + ' Sample')

filename = Dataset + '_b_efficiency_eta' + filename_bw + '.png'

In [ ]:
SaveDirectory = maindirectory + '/FlavorTagEfficiencies/images/1D/btags_eta/' # split histograms into subdirectories
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
if PlotWithLargeBins == True:
    binwidth = '_manualbins'
    filename_bw = '_large_bins'
else:
    binwidth = ''
    filename_bw = ''
for dataset,output in Datasets.items(): 
    plt.rcParams.update({
    'font.size': 14,
    'axes.titlesize': 18,
    'axes.labelsize': 18,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12
    })
    fig, (ax01, ax02, ax11, ax12) = plt.subplots(1, 4, figsize=(17,5), sharex=True)

    b_eff_numerator_eta_s01 = output['b_eff_numerator_s01'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    b_eff_denominator_eta_s01 = output['b_eff_denominator_s01'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    
    
    b_eff_pt_s01 = hist.plotratio(num = b_eff_numerator_eta_s01, denom = b_eff_denominator_eta_s01, 
                                  ax=ax01, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                                  unc = 'num')

    
    ax01.set_title(r'SubJet$_{01}\left(\eta\right)$')
    ax01.set_ylabel('b-Tag Efficiency')
    ax01.set_ylim(bottom=0., top=1.)

    # ------------------------------------------------------------------------------ #

    b_eff_numerator_eta_s02 = output['b_eff_numerator_s02'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    b_eff_denominator_eta_s02 = output['b_eff_denominator_s02'+binwidth].integrate('subjetpt').integrate('dataset', dataset)

    
    b_eff_pt_s02 = hist.plotratio(num = b_eff_numerator_eta_s02, denom = b_eff_denominator_eta_s02, 
                                  ax=ax02, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                                  unc = 'num')
    
    
    ax02.set_title(r'SubJet$_{02}\left(\eta\right)$')
    ax02.set_ylabel('')
    ax02.set_ylim(bottom=0., top=1.)

    # ------------------------------------------------------------------------------ #

    b_eff_numerator_eta_s11 = output['b_eff_numerator_s11'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    b_eff_denominator_eta_s11 = output['b_eff_denominator_s11'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    
    
    b_eff_pt_s11 = hist.plotratio(num = b_eff_numerator_eta_s11, denom = b_eff_denominator_eta_s11, 
                                  ax=ax11, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                                  unc = 'num')
    
    
    ax11.set_title(r'SubJet$_{11}\left(eta\right)$')
    ax11.set_ylabel('')
    ax11.set_ylim(bottom=0., top=1.)

    # ------------------------------------------------------------------------------ #

    b_eff_numerator_eta_s12 = output['b_eff_numerator_s12'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    b_eff_denominator_eta_s12 = output['b_eff_denominator_s12'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    
    
    b_eff_pt_s12 = hist.plotratio(num = b_eff_numerator_eta_s12, denom = b_eff_denominator_eta_s12, 
                                  ax=ax12, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                                  unc = 'num')
    
    
    ax12.set_title(r'SubJet$_{12}\left(eta\right)$')
    ax12.set_ylabel('')
    ax12.set_ylim(bottom=0., top=1.)
    leg = ax11.legend(labels=[r'$b_{eff}$'], bbox_to_anchor=(0.7, 1.4), loc='upper right')
    fig.suptitle(dataset + ' Sample')

    filename = dataset + '_b_efficiency_eta' + filename_bw + '.png'
    print(filename + ' generated')
    plt.savefig(SaveDirectory+filename, bbox_inches="tight")

# Plot (and save) the c quark tagging efficiencies as functions of $p_T$ 

In [ ]:
SaveDirectory = maindirectory + '/FlavorTagEfficiencies/images/1D/ctags_pt/' # split histograms into subdirectories
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
if PlotWithLargeBins == True:
    binwidth = '_manualbins'
    filename_bw = '_large_bins'
else:
    binwidth = ''
    filename_bw = ''

plt.rcParams.update({
'font.size': 14,
'axes.titlesize': 18,
'axes.labelsize': 18,
'xtick.labelsize': 12,
'ytick.labelsize': 12
})
fig, (ax01, ax02, ax11, ax12) = plt.subplots(1, 4, figsize=(17,5), sharex=True)

c_eff_numerator_pt_s01_biased1 = TTbar_biased_700to1000_unweighted['c_eff_numerator_s01'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
c_eff_numerator_pt_s01_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_numerator_s01'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')
c_eff_denominator_pt_s01_biased1 = TTbar_biased_700to1000_unweighted['c_eff_denominator_s01'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
c_eff_denominator_pt_s01_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_denominator_s01'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')

c_eff_numerator_pt_s01 = c_eff_numerator_pt_s01_biased1.copy()
c_eff_numerator_pt_s01.add(c_eff_numerator_pt_s01_biased2)
c_eff_denominator_pt_s01 = c_eff_denominator_pt_s01_biased1.copy()
c_eff_denominator_pt_s01.add(c_eff_denominator_pt_s01_biased2)

c_eff_pt_s01 = hist.plotratio(num = c_eff_numerator_pt_s01, denom = c_eff_denominator_pt_s01, 
                              ax=ax01, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                              unc = 'num')


ax01.set_title(r'SubJet$_{01}\left(p_T\right)$')
ax01.set_ylabel('c-Tag Efficiency')
ax01.set_ylim(bottom=0., top=1.)

# ------------------------------------------------------------------------------ #

c_eff_numerator_pt_s02_biased1 = TTbar_biased_700to1000_unweighted['c_eff_numerator_s02'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
c_eff_numerator_pt_s02_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_numerator_s02'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')
c_eff_denominator_pt_s02_biased1 = TTbar_biased_700to1000_unweighted['c_eff_denominator_s02'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
c_eff_denominator_pt_s02_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_denominator_s02'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')

c_eff_numerator_pt_s02 = c_eff_numerator_pt_s02_biased1.copy()
c_eff_numerator_pt_s02.add(c_eff_numerator_pt_s02_biased2)
c_eff_denominator_pt_s02 = c_eff_denominator_pt_s02_biased1.copy()
c_eff_denominator_pt_s02.add(c_eff_denominator_pt_s02_biased2) 

c_eff_pt_s02 = hist.plotratio(num = c_eff_numerator_pt_s02, denom = c_eff_denominator_pt_s02, 
                              ax=ax02, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                              unc = 'num')


ax02.set_title(r'SubJet$_{02}\left(p_T\right)$')
ax02.set_ylabel('')
ax02.set_ylim(bottom=0., top=1.)

# # ------------------------------------------------------------------------------ #

c_eff_numerator_pt_s11_biased1 = TTbar_biased_700to1000_unweighted['c_eff_numerator_s11'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
c_eff_numerator_pt_s11_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_numerator_s11'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')
c_eff_denominator_pt_s11_biased1 = TTbar_biased_700to1000_unweighted['c_eff_denominator_s11'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
c_eff_denominator_pt_s11_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_denominator_s11'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')

c_eff_numerator_pt_s11 = c_eff_numerator_pt_s11_biased1.copy()
c_eff_numerator_pt_s11.add(c_eff_numerator_pt_s11_biased2)
c_eff_denominator_pt_s11 = c_eff_denominator_pt_s11_biased1.copy()
c_eff_denominator_pt_s11.add(c_eff_denominator_pt_s11_biased2)

c_eff_pt_s11 = hist.plotratio(num = c_eff_numerator_pt_s11, denom = c_eff_denominator_pt_s11, 
                              ax=ax11, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                              unc = 'num')


ax11.set_title(r'SubJet$_{11}\left(p_T\right)$')
ax11.set_ylabel('')
ax11.set_ylim(bottom=0., top=1.)

# # ------------------------------------------------------------------------------ #

c_eff_numerator_pt_s12_biased1 = TTbar_biased_700to1000_unweighted['c_eff_numerator_s12'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
c_eff_numerator_pt_s12_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_numerator_s12'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')
c_eff_denominator_pt_s12_biased1 = TTbar_biased_700to1000_unweighted['c_eff_denominator_s12'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
c_eff_denominator_pt_s12_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_denominator_s12'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')

c_eff_numerator_pt_s12 = c_eff_numerator_pt_s12_biased1.copy()
c_eff_numerator_pt_s12.add(c_eff_numerator_pt_s12_biased2)
c_eff_denominator_pt_s12 = c_eff_denominator_pt_s12_biased1.copy()
c_eff_denominator_pt_s12.add(c_eff_denominator_pt_s12_biased2)

c_eff_pt_s12 = hist.plotratio(num = c_eff_numerator_pt_s12, denom = c_eff_denominator_pt_s12, 
                              ax=ax12, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                              unc = 'num')


ax12.set_title(r'SubJet$_{12}\left(p_T\right)$')
ax12.set_ylabel('')
ax12.set_ylim(bottom=0., top=1.)
leg = ax11.legend(labels=[r'$c_{eff}$'], bbox_to_anchor=(0.7, 1.4), loc='upper right')

Dataset = 'TTbarBiasedSum'
fig.suptitle(Dataset + ' Sample')

filename = Dataset + '_c_efficiency_pt' + filename_bw + '.png'

In [ ]:
SaveDirectory = maindirectory + '/FlavorTagEfficiencies/images/1D/ctags_pt/' # split histograms into subdirectories
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
if PlotWithLargeBins == True:
    binwidth = '_manualbins'
    filename_bw = '_large_bins'
else:
    binwidth = ''
    filename_bw = ''
for dataset,output in Datasets.items(): 
    plt.rcParams.update({
    'font.size': 14,
    'axes.titlesize': 18,
    'axes.labelsize': 18,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12
    })
    fig, (ax01, ax02, ax11, ax12) = plt.subplots(1, 4, figsize=(17,5), sharex=True)
    
    c_eff_numerator_pt_s01 = output['c_eff_numerator_s01'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    c_eff_denominator_pt_s01 = output['c_eff_denominator_s01'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    
    
    c_eff_pt_s01 = hist.plotratio(num = c_eff_numerator_pt_s01, denom = c_eff_denominator_pt_s01, 
                                  ax=ax01, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                                  unc = 'num')
    
    ax01.set_title(r'SubJet$_{01}\left(p_T\right)$')
    ax01.set_ylabel('c-Tag Efficiency')
    ax01.set_ylim(bottom=0., top=1.)

    # ------------------------------------------------------------------------------ #

    c_eff_numerator_pt_s02 = output['c_eff_numerator_s02'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    c_eff_denominator_pt_s02 = output['c_eff_denominator_s02'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    
    
    c_eff_pt_s02 = hist.plotratio(num = c_eff_numerator_pt_s02, denom = c_eff_denominator_pt_s02, 
                                  ax=ax02, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                                  unc = 'num')
    
    ax02.set_title(r'SubJet$_{02}\left(p_T\right)$')
    ax02.set_ylabel('')
    ax02.set_ylim(bottom=0., top=1.)

    # ------------------------------------------------------------------------------ #

    c_eff_numerator_pt_s11 = output['c_eff_numerator_s11'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    c_eff_denominator_pt_s11 = output['c_eff_denominator_s11'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    
    
    c_eff_pt_s11 = hist.plotratio(num = c_eff_numerator_pt_s11, denom = c_eff_denominator_pt_s11, 
                                  ax=ax11, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                                  unc = 'num')
    
    
    ax11.set_title(r'SubJet$_{11}\left(p_T\right)$')
    ax11.set_ylabel('')
    ax11.set_ylim(bottom=0., top=1.)

    # ------------------------------------------------------------------------------ #

    c_eff_numerator_pt_s12 = output['c_eff_numerator_s12'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    c_eff_denominator_pt_s12 = output['c_eff_denominator_s12'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    
    
    c_eff_pt_s12 = hist.plotratio(num = c_eff_numerator_pt_s12, denom = c_eff_denominator_pt_s12, 
                                  ax=ax12, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                                  unc = 'num')
    
    
    ax12.set_title(r'SubJet$_{12}\left(p_T\right)$')
    ax12.set_ylabel('')
    ax12.set_ylim(bottom=0., top=1.)
    leg = ax11.legend(labels=[r'$c_{eff}$'], bbox_to_anchor=(0.7, 1.4), loc='upper right')
    fig.suptitle(dataset + ' Sample')

    filename = dataset + '_c_efficiency_pt' + filename_bw + '.png'
    print(filename + ' generated')
    plt.savefig(SaveDirectory+filename, bbox_inches="tight")

# Plot (and save) the c quark tagging efficiencies as functions of $\eta$ 

In [ ]:
SaveDirectory = maindirectory + '/FlavorTagEfficiencies/images/1D/ctags_eta/' # split histograms into subdirectories
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
if PlotWithLargeBins == True:
    binwidth = '_manualbins'
    filename_bw = '_large_bins'
else:
    binwidth = ''
    filename_bw = ''

plt.rcParams.update({
'font.size': 14,
'axes.titlesize': 18,
'axes.labelsize': 18,
'xtick.labelsize': 12,
'ytick.labelsize': 12
})
fig, (ax01, ax02, ax11, ax12) = plt.subplots(1, 4, figsize=(17,5), sharex=True)

c_eff_numerator_eta_s01_biased1 = TTbar_biased_700to1000_unweighted['c_eff_numerator_s01'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
c_eff_numerator_eta_s01_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_numerator_s01'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')
c_eff_denominator_eta_s01_biased1 = TTbar_biased_700to1000_unweighted['c_eff_denominator_s01'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
c_eff_denominator_eta_s01_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_denominator_s01'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')

c_eff_numerator_eta_s01 = c_eff_numerator_eta_s01_biased1.copy()
c_eff_numerator_eta_s01.add(c_eff_numerator_eta_s01_biased2)
c_eff_denominator_eta_s01 = c_eff_denominator_eta_s01_biased1.copy()
c_eff_denominator_eta_s01.add(c_eff_denominator_eta_s01_biased2)

c_eff_eta_s01 = hist.plotratio(num = c_eff_numerator_eta_s01, denom = c_eff_denominator_eta_s01, 
                              ax=ax01, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                              unc = 'num')


ax01.set_title(r'SubJet$_{01}\left(\eta\right)$')
ax01.set_ylabel('c-Tag Efficiency')
ax01.set_ylim(bottom=0., top=1.)

# ------------------------------------------------------------------------------ #

c_eff_numerator_eta_s02_biased1 = TTbar_biased_700to1000_unweighted['c_eff_numerator_s02'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
c_eff_numerator_eta_s02_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_numerator_s02'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')
c_eff_denominator_eta_s02_biased1 = TTbar_biased_700to1000_unweighted['c_eff_denominator_s02'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
c_eff_denominator_eta_s02_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_denominator_s02'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')

c_eff_numerator_eta_s02 = c_eff_numerator_eta_s02_biased1.copy()
c_eff_numerator_eta_s02.add(c_eff_numerator_eta_s02_biased2)
c_eff_denominator_eta_s02 = c_eff_denominator_eta_s02_biased1.copy()
c_eff_denominator_eta_s02.add(c_eff_denominator_eta_s02_biased2)

c_eff_eta_s02 = hist.plotratio(num = c_eff_numerator_eta_s02, denom = c_eff_denominator_eta_s02, 
                              ax=ax02, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                              unc = 'num')


ax02.set_title(r'SubJet$_{02}\left(\eta\right)$')
ax02.set_ylabel('')
ax02.set_ylim(bottom=0., top=1.)

# # ------------------------------------------------------------------------------ #

c_eff_numerator_eta_s11_biased1 = TTbar_biased_700to1000_unweighted['c_eff_numerator_s11'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
c_eff_numerator_eta_s11_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_numerator_s11'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')
c_eff_denominator_eta_s11_biased1 = TTbar_biased_700to1000_unweighted['c_eff_denominator_s11'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
c_eff_denominator_eta_s11_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_denominator_s11'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')

c_eff_numerator_eta_s11 = c_eff_numerator_eta_s11_biased1.copy()
c_eff_numerator_eta_s11.add(c_eff_numerator_eta_s11_biased2)
c_eff_denominator_eta_s11 = c_eff_denominator_eta_s11_biased1.copy()
c_eff_denominator_eta_s11.add(c_eff_denominator_eta_s11_biased2)

c_eff_eta_s11 = hist.plotratio(num = c_eff_numerator_eta_s11, denom = c_eff_denominator_eta_s11, 
                              ax=ax11, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                              unc = 'num')


ax11.set_title(r'SubJet$_{11}\left(\eta\right)$')
ax11.set_ylabel('')
ax11.set_ylim(bottom=0., top=1.)

# # ------------------------------------------------------------------------------ #

c_eff_numerator_eta_s12_biased1 = TTbar_biased_700to1000_unweighted['c_eff_numerator_s12'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
c_eff_numerator_eta_s12_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_numerator_s12'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')
c_eff_denominator_eta_s12_biased1 = TTbar_biased_700to1000_unweighted['c_eff_denominator_s12'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
c_eff_denominator_eta_s12_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_denominator_s12'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')

c_eff_numerator_eta_s12 = c_eff_numerator_eta_s12_biased1.copy()
c_eff_numerator_eta_s12.add(c_eff_numerator_eta_s12_biased2)
c_eff_denominator_eta_s12 = c_eff_denominator_eta_s12_biased1.copy()
c_eff_denominator_eta_s12.add(c_eff_denominator_eta_s12_biased2)

c_eff_eta_s12 = hist.plotratio(num = c_eff_numerator_eta_s12, denom = c_eff_denominator_eta_s12, 
                              ax=ax12, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                              unc = 'num')


ax12.set_title(r'SubJet$_{12}\left(\eta\right)$')
ax12.set_ylabel('')
ax12.set_ylim(bottom=0., top=1.)
leg = ax11.legend(labels=[r'$c_{eff}$'], bbox_to_anchor=(0.7, 1.4), loc='upper right')

Dataset = 'TTbarBiasedSum'
fig.suptitle(Dataset + ' Sample')

filename = Dataset + '_c_efficiency_eta' + filename_bw + '.png'

In [ ]:
SaveDirectory = maindirectory + '/FlavorTagEfficiencies/images/1D/ctags_eta/' # split histograms into subdirectories
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
if PlotWithLargeBins == True:
    binwidth = '_manualbins'
    filename_bw = '_large_bins'
else:
    binwidth = ''
    filename_bw = ''
for dataset,output in Datasets.items(): 
    plt.rcParams.update({
    'font.size': 14,
    'axes.titlesize': 18,
    'axes.labelsize': 18,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12
    })
    fig, (ax01, ax02, ax11, ax12) = plt.subplots(1, 4, figsize=(17,5), sharex=True)
    
    c_eff_numerator_eta_s01 = output['c_eff_numerator_s01'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    c_eff_denominator_eta_s01 = output['c_eff_denominator_s01'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    
    
    c_eff_pt_s01 = hist.plotratio(num = c_eff_numerator_eta_s01, denom = c_eff_denominator_eta_s01, 
                                  ax=ax01, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                                  unc = 'num')
    
    ax01.set_title(r'SubJet$_{01}\left(\eta\right)$')
    ax01.set_ylabel('c-Tag Efficiency')
    ax01.set_ylim(bottom=0., top=1.)

    # ------------------------------------------------------------------------------ #

    c_eff_numerator_eta_s02 = output['c_eff_numerator_s02'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    c_eff_denominator_eta_s02 = output['c_eff_denominator_s02'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    
    
    c_eff_pt_s02 = hist.plotratio(num = c_eff_numerator_eta_s02, denom = c_eff_denominator_eta_s02, 
                                  ax=ax02, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                                  unc = 'num')
    
    ax02.set_title(r'SubJet$_{02}\left(\eta\right)$')
    ax02.set_ylabel('')
    ax02.set_ylim(bottom=0., top=1.)

    # ------------------------------------------------------------------------------ #

    c_eff_numerator_eta_s11 = output['c_eff_numerator_s11'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    c_eff_denominator_eta_s11 = output['c_eff_denominator_s11'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    
    
    c_eff_pt_s11 = hist.plotratio(num = c_eff_numerator_eta_s11, denom = c_eff_denominator_eta_s11, 
                                  ax=ax11, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                                  unc = 'num')
    
    
    ax11.set_title(r'SubJet$_{11}\left(\eta\right)$')
    ax11.set_ylabel('')
    ax11.set_ylim(bottom=0., top=1.)

    # ------------------------------------------------------------------------------ #

    c_eff_numerator_eta_s12 = output['c_eff_numerator_s12'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    c_eff_denominator_eta_s12 = output['c_eff_denominator_s12'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    
    
    c_eff_pt_s12 = hist.plotratio(num = c_eff_numerator_eta_s12, denom = c_eff_denominator_eta_s12, 
                                  ax=ax12, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                                  unc = 'num')
    
    
    ax12.set_title(r'SubJet$_{12}\left(\eta\right)$')
    ax12.set_ylabel('')
    ax12.set_ylim(bottom=0., top=1.)
    leg = ax11.legend(labels=[r'$c_{eff}$'], bbox_to_anchor=(0.7, 1.4), loc='upper right')
    fig.suptitle(dataset + ' Sample')

    filename = dataset + '_c_efficiency_eta' + filename_bw + '.png'
    print(filename + ' generated')
    plt.savefig(SaveDirectory+filename, bbox_inches="tight")

# Plot (and save) the light quark tagging efficiencies as functions of $p_T$

In [ ]:
SaveDirectory = maindirectory + '/FlavorTagEfficiencies/images/1D/udsgtags_pt/' # split histograms into subdirectories
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
if PlotWithLargeBins == True:
    binwidth = '_manualbins'
    filename_bw = '_large_bins'
else:
    binwidth = ''
    filename_bw = ''

plt.rcParams.update({
'font.size': 14,
'axes.titlesize': 18,
'axes.labelsize': 18,
'xtick.labelsize': 12,
'ytick.labelsize': 12
})
fig, (ax01, ax02, ax11, ax12) = plt.subplots(1, 4, figsize=(17,5), sharex=True)

udsg_eff_numerator_pt_s01_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_numerator_s01'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_numerator_pt_s01_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_numerator_s01'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')
udsg_eff_denominator_pt_s01_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_denominator_s01'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_denominator_pt_s01_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_denominator_s01'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')

udsg_eff_numerator_pt_s01 = udsg_eff_numerator_pt_s01_biased1.copy()
udsg_eff_numerator_pt_s01.add(udsg_eff_numerator_pt_s01_biased2)
udsg_eff_denominator_pt_s01 = udsg_eff_denominator_pt_s01_biased1.copy()
udsg_eff_denominator_pt_s01.add(udsg_eff_denominator_pt_s01_biased2)

udsg_eff_pt_s01 = hist.plotratio(num = udsg_eff_numerator_pt_s01, denom = udsg_eff_denominator_pt_s01, 
                              ax=ax01, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                              unc = 'num')


ax01.set_title(r'SubJet$_{01}\left(p_T\right)$')
ax01.set_ylabel('udsg-Tag Efficiency')
ax01.set_ylim(bottom=0., top=1.)

# ------------------------------------------------------------------------------ #

udsg_eff_numerator_pt_s02_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_numerator_s02'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_numerator_pt_s02_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_numerator_s02'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')
udsg_eff_denominator_pt_s02_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_denominator_s02'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_denominator_pt_s02_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_denominator_s02'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')

udsg_eff_numerator_pt_s02 = udsg_eff_numerator_pt_s02_biased1.copy()
udsg_eff_numerator_pt_s02.add(udsg_eff_numerator_pt_s02_biased2)
udsg_eff_denominator_pt_s02 = udsg_eff_denominator_pt_s02_biased1.copy()
udsg_eff_denominator_pt_s02.add(udsg_eff_denominator_pt_s02_biased2) 

udsg_eff_pt_s02 = hist.plotratio(num = udsg_eff_numerator_pt_s02, denom = udsg_eff_denominator_pt_s02, 
                              ax=ax02, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                              unc = 'num')


ax02.set_title(r'SubJet$_{02}\left(p_T\right)$')
ax02.set_ylabel('')
ax02.set_ylim(bottom=0., top=1.)

# # ------------------------------------------------------------------------------ #

udsg_eff_numerator_pt_s11_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_numerator_s11'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_numerator_pt_s11_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_numerator_s11'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')
udsg_eff_denominator_pt_s11_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_denominator_s11'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_denominator_pt_s11_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_denominator_s11'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')

udsg_eff_numerator_pt_s11 = udsg_eff_numerator_pt_s11_biased1.copy()
udsg_eff_numerator_pt_s11.add(udsg_eff_numerator_pt_s11_biased2)
udsg_eff_denominator_pt_s11 = udsg_eff_denominator_pt_s11_biased1.copy()
udsg_eff_denominator_pt_s11.add(udsg_eff_denominator_pt_s11_biased2)

udsg_eff_pt_s11 = hist.plotratio(num = udsg_eff_numerator_pt_s11, denom = udsg_eff_denominator_pt_s11, 
                              ax=ax11, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                              unc = 'num')


ax11.set_title(r'SubJet$_{11}\left(p_T\right)$')
ax11.set_ylabel('')
ax11.set_ylim(bottom=0., top=1.)

# # ------------------------------------------------------------------------------ #

udsg_eff_numerator_pt_s12_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_numerator_s12'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_numerator_pt_s12_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_numerator_s12'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')
udsg_eff_denominator_pt_s12_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_denominator_s12'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_denominator_pt_s12_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_denominator_s12'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')

udsg_eff_numerator_pt_s12 = udsg_eff_numerator_pt_s12_biased1.copy()
udsg_eff_numerator_pt_s12.add(udsg_eff_numerator_pt_s12_biased2)
udsg_eff_denominator_pt_s12 = udsg_eff_denominator_pt_s12_biased1.copy()
udsg_eff_denominator_pt_s12.add(udsg_eff_denominator_pt_s12_biased2)

udsg_eff_pt_s12 = hist.plotratio(num = udsg_eff_numerator_pt_s12, denom = udsg_eff_denominator_pt_s12, 
                              ax=ax12, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                              unc = 'num')


ax12.set_title(r'SubJet$_{12}\left(p_T\right)$')
ax12.set_ylabel('')
ax12.set_ylim(bottom=0., top=1.)
leg = ax11.legend(labels=[r'$c_{eff}$'], bbox_to_anchor=(0.7, 1.4), loc='upper right')

Dataset = 'TTbarBiasedSum'
fig.suptitle(Dataset + ' Sample')

filename = Dataset + '_udsg_efficiency_pt' + filename_bw + '.png'

In [ ]:
SaveDirectory = maindirectory + '/FlavorTagEfficiencies/images/1D/udsgtags_pt/' # split histograms into subdirectories
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
if PlotWithLargeBins == True:
    binwidth = '_manualbins'
    filename_bw = '_large_bins'
else:
    binwidth = ''
    filename_bw = ''
for dataset,output in Datasets.items(): 
    plt.rcParams.update({
    'font.size': 14,
    'axes.titlesize': 18,
    'axes.labelsize': 18,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12
    })
    fig, (ax01, ax02, ax11, ax12) = plt.subplots(1, 4, figsize=(17,5), sharex=True)

    udsg_eff_numerator_pt_s01 = output['udsg_eff_numerator_s01'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    udsg_eff_denominator_pt_s01 = output['udsg_eff_denominator_s01'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    
    
    udsg_eff_pt_s01 = hist.plotratio(num = udsg_eff_numerator_pt_s01, denom = udsg_eff_denominator_pt_s01, 
                                  ax=ax01, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                                  unc = 'num')
    
    ax01.set_title(r'SubJet$_{01}\left(p_T\right)$')
    ax01.set_ylabel('udsg-Tag Efficiency')
    ax01.set_ylim(bottom=0., top=1.)

    # ------------------------------------------------------------------------------ #

    udsg_eff_numerator_pt_s02 = output['udsg_eff_numerator_s02'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    udsg_eff_denominator_pt_s02 = output['udsg_eff_denominator_s02'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    
    
    udsg_eff_pt_s02 = hist.plotratio(num = udsg_eff_numerator_pt_s02, denom = udsg_eff_denominator_pt_s02, 
                                  ax=ax02, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                                  unc = 'num')
    
    ax02.set_title(r'SubJet$_{02}\left(p_T\right)$')
    ax02.set_ylabel('')
    ax02.set_ylim(bottom=0., top=1.)

    # ------------------------------------------------------------------------------ #

    udsg_eff_numerator_pt_s11 = output['udsg_eff_numerator_s11'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    udsg_eff_denominator_pt_s11 = output['udsg_eff_denominator_s11'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    
    
    udsg_eff_pt_s11 = hist.plotratio(num = udsg_eff_numerator_pt_s11, denom = udsg_eff_denominator_pt_s11, 
                                  ax=ax11, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                                  unc = 'num')
    
    ax11.set_title(r'SubJet$_{11}\left(p_T\right)$')
    ax11.set_ylabel('')
    ax11.set_ylim(bottom=0., top=1.)

    # ------------------------------------------------------------------------------ #

    udsg_eff_numerator_pt_s12 = output['udsg_eff_numerator_s12'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    udsg_eff_denominator_pt_s12 = output['udsg_eff_denominator_s12'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    
    
    udsg_eff_pt_s12 = hist.plotratio(num = udsg_eff_numerator_pt_s12, denom = udsg_eff_denominator_pt_s12, 
                                  ax=ax12, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                                  unc = 'num')
    
    ax12.set_title(r'SubJet$_{12}\left(p_T\right)$')
    ax12.set_ylabel('')
    ax12.set_ylim(bottom=0., top=1.)
    leg = ax11.legend(labels=[r'$udsg_{eff}$'], bbox_to_anchor=(0.7, 1.4), loc='upper right')
    fig.suptitle(dataset + ' Sample')

    filename = dataset + '_udsg_efficiency_pt' + filename_bw + '.png'
    print(filename + ' generated')
    plt.savefig(SaveDirectory+filename, bbox_inches="tight")

# Plot (and save) the light quark tagging efficiencies as functions of $\eta$

In [ ]:
SaveDirectory = maindirectory + '/FlavorTagEfficiencies/images/1D/udsgtags_eta/' # split histograms into subdirectories
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
if PlotWithLargeBins == True:
    binwidth = '_manualbins'
    filename_bw = '_large_bins'
else:
    binwidth = ''
    filename_bw = ''

plt.rcParams.update({
'font.size': 14,
'axes.titlesize': 18,
'axes.labelsize': 18,
'xtick.labelsize': 12,
'ytick.labelsize': 12
})
fig, (ax01, ax02, ax11, ax12) = plt.subplots(1, 4, figsize=(17,5), sharex=True)

udsg_eff_numerator_eta_s01_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_numerator_s01'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_numerator_eta_s01_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_numerator_s01'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')
udsg_eff_denominator_eta_s01_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_denominator_s01'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_denominator_eta_s01_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_denominator_s01'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')

udsg_eff_numerator_eta_s01 = udsg_eff_numerator_eta_s01_biased1.copy()
udsg_eff_numerator_eta_s01.add(udsg_eff_numerator_eta_s01_biased2)
udsg_eff_denominator_eta_s01 = udsg_eff_denominator_eta_s01_biased1.copy()
udsg_eff_denominator_eta_s01.add(udsg_eff_denominator_eta_s01_biased2)

udsg_eff_eta_s01 = hist.plotratio(num = udsg_eff_numerator_eta_s01, denom = udsg_eff_denominator_eta_s01, 
                              ax=ax01, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                              unc = 'num')


ax01.set_title(r'SubJet$_{01}\left(\eta\right)$')
ax01.set_ylabel('udsg-Tag Efficiency')
ax01.set_ylim(bottom=0., top=1.)

# ------------------------------------------------------------------------------ #

udsg_eff_numerator_eta_s02_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_numerator_s02'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_numerator_eta_s02_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_numerator_s02'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')
udsg_eff_denominator_eta_s02_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_denominator_s02'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_denominator_eta_s02_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_denominator_s02'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')

udsg_eff_numerator_eta_s02 = udsg_eff_numerator_eta_s02_biased1.copy()
udsg_eff_numerator_eta_s02.add(udsg_eff_numerator_eta_s02_biased2)
udsg_eff_denominator_eta_s02 = udsg_eff_denominator_eta_s02_biased1.copy()
udsg_eff_denominator_eta_s02.add(udsg_eff_denominator_eta_s02_biased2)

udsg_eff_eta_s02 = hist.plotratio(num = udsg_eff_numerator_eta_s02, denom = udsg_eff_denominator_eta_s02, 
                              ax=ax02, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                              unc = 'num')


ax02.set_title(r'SubJet$_{02}\left(\eta\right)$')
ax02.set_ylabel('')
ax02.set_ylim(bottom=0., top=1.)

# # ------------------------------------------------------------------------------ #

udsg_eff_numerator_eta_s11_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_numerator_s11'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_numerator_eta_s11_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_numerator_s11'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')
udsg_eff_denominator_eta_s11_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_denominator_s11'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_denominator_eta_s11_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_denominator_s11'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')

udsg_eff_numerator_eta_s11 = udsg_eff_numerator_eta_s11_biased1.copy()
udsg_eff_numerator_eta_s11.add(udsg_eff_numerator_eta_s11_biased2)
udsg_eff_denominator_eta_s11 = udsg_eff_denominator_eta_s11_biased1.copy()
udsg_eff_denominator_eta_s11.add(udsg_eff_denominator_eta_s11_biased2)

udsg_eff_eta_s11 = hist.plotratio(num = udsg_eff_numerator_eta_s11, denom = udsg_eff_denominator_eta_s11, 
                              ax=ax11, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                              unc = 'num')


ax11.set_title(r'SubJet$_{11}\left(\eta\right)$')
ax11.set_ylabel('')
ax11.set_ylim(bottom=0., top=1.)

# # ------------------------------------------------------------------------------ #

udsg_eff_numerator_eta_s12_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_numerator_s12'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_numerator_eta_s12_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_numerator_s12'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')
udsg_eff_denominator_eta_s12_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_denominator_s12'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_denominator_eta_s12_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_denominator_s12'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')

udsg_eff_numerator_eta_s12 = udsg_eff_numerator_eta_s12_biased1.copy()
udsg_eff_numerator_eta_s12.add(udsg_eff_numerator_eta_s12_biased2)
udsg_eff_denominator_eta_s12 = udsg_eff_denominator_eta_s12_biased1.copy()
udsg_eff_denominator_eta_s12.add(udsg_eff_denominator_eta_s12_biased2)

udsg_eff_eta_s12 = hist.plotratio(num = udsg_eff_numerator_eta_s12, denom = udsg_eff_denominator_eta_s12, 
                              ax=ax12, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                              unc = 'num')


ax12.set_title(r'SubJet$_{12}\left(\eta\right)$')
ax12.set_ylabel('')
ax12.set_ylim(bottom=0., top=1.)
leg = ax11.legend(labels=[r'$udsg_{eff}$'], bbox_to_anchor=(0.7, 1.4), loc='upper right')

Dataset = 'TTbarBiasedSum'
fig.suptitle(Dataset + ' Sample')

filename = Dataset + '_udsg_efficiency_eta' + filename_bw + '.png'

In [ ]:
SaveDirectory = maindirectory + '/FlavorTagEfficiencies/images/1D/udsgtags_eta/' # split histograms into subdirectories
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
if PlotWithLargeBins == True:
    binwidth = '_manualbins'
    filename_bw = '_large_bins'
else:
    binwidth = ''
    filename_bw = ''
for dataset,output in Datasets.items(): 
    plt.rcParams.update({
    'font.size': 14,
    'axes.titlesize': 18,
    'axes.labelsize': 18,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12
    })
    fig, (ax01, ax02, ax11, ax12) = plt.subplots(1, 4, figsize=(17,5), sharex=True)

    udsg_eff_numerator_eta_s01 = output['udsg_eff_numerator_s01'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    udsg_eff_denominator_eta_s01 = output['udsg_eff_denominator_s01'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    
    
    udsg_eff_pt_s01 = hist.plotratio(num = udsg_eff_numerator_eta_s01, denom = udsg_eff_denominator_eta_s01, 
                                  ax=ax01, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                                  unc = 'num')
    
    ax01.set_title(r'SubJet$_{01}\left(\eta\right)$')
    ax01.set_ylabel('b-Tag Efficiency')
    ax01.set_ylim(bottom=0., top=1.)

    # ------------------------------------------------------------------------------ #

    udsg_eff_numerator_eta_s02 = output['udsg_eff_numerator_s02'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    udsg_eff_denominator_eta_s02 = output['udsg_eff_denominator_s02'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    
    
    udsg_eff_pt_s02 = hist.plotratio(num = udsg_eff_numerator_eta_s02, denom = udsg_eff_denominator_eta_s02, 
                                  ax=ax02, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                                  unc = 'num')
    
    ax02.set_title(r'SubJet$_{02}\left(\eta\right)$')
    ax02.set_ylabel('')
    ax02.set_ylim(bottom=0., top=1.)

    # ------------------------------------------------------------------------------ #

    udsg_eff_numerator_eta_s11 = output['udsg_eff_numerator_s11'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    udsg_eff_denominator_eta_s11 = output['udsg_eff_denominator_s11'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    
    
    udsg_eff_pt_s11 = hist.plotratio(num = udsg_eff_numerator_eta_s11, denom = udsg_eff_denominator_eta_s11, 
                                  ax=ax11, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                                  unc = 'num')
    
    ax11.set_title(r'SubJet$_{11}\left(\eta\right)$')
    ax11.set_ylabel('')
    ax11.set_ylim(bottom=0., top=1.)

    # ------------------------------------------------------------------------------ #

    udsg_eff_numerator_eta_s12 = output['udsg_eff_numerator_s12'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    udsg_eff_denominator_eta_s12 = output['udsg_eff_denominator_s12'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    
    
    udsg_eff_pt_s12 = hist.plotratio(num = udsg_eff_numerator_eta_s12, denom = udsg_eff_denominator_eta_s12, 
                                  ax=ax12, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                                  unc = 'num')
    
    ax12.set_title(r'SubJet$_{12}\left(\eta\right)$')
    ax12.set_ylabel('')
    ax12.set_ylim(bottom=0., top=1.)
    leg = ax11.legend(labels=[r'$udsg_{eff}$'], bbox_to_anchor=(0.7, 1.4), loc='upper right')
    fig.suptitle(dataset + ' Sample')

    filename = dataset + '_udsg_efficiency_eta' + filename_bw + '.png'
    print(filename + ' generated')
    plt.savefig(SaveDirectory+filename, bbox_inches="tight")

# Plot (and save) the flavor tagging efficiencies as functions of $p_T$ 

In [ ]:
SaveDirectory = maindirectory + '/FlavorTagEfficiencies/images/1D/flavortags_pt/' # split histograms into subdirectories
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
if PlotWithLargeBins == True:
    binwidth = '_manualbins'
    filename_bw = '_large_bins'
else:
    binwidth = ''
    filename_bw = ''

plt.rcParams.update({
'font.size': 14,
'axes.titlesize': 18,
'axes.labelsize': 18,
'xtick.labelsize': 12,
'ytick.labelsize': 12
})
fig, (ax01, ax02, ax11, ax12) = plt.subplots(1, 4, figsize=(17,5), sharex=True)

b_eff_numerator_pt_s01_biased1 = TTbar_biased_700to1000_unweighted['b_eff_numerator_s01'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
b_eff_numerator_pt_s01_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_numerator_s01'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')
b_eff_denominator_pt_s01_biased1 = TTbar_biased_700to1000_unweighted['b_eff_denominator_s01'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
b_eff_denominator_pt_s01_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_denominator_s01'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')

b_eff_numerator_pt_s01 = b_eff_numerator_pt_s01_biased1.copy()
b_eff_numerator_pt_s01.add(b_eff_numerator_pt_s01_biased2)
b_eff_denominator_pt_s01 = b_eff_denominator_pt_s01_biased1.copy()
b_eff_denominator_pt_s01.add(b_eff_denominator_pt_s01_biased2)

c_eff_numerator_pt_s01_biased1 = TTbar_biased_700to1000_unweighted['c_eff_numerator_s01'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
c_eff_numerator_pt_s01_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_numerator_s01'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')
c_eff_denominator_pt_s01_biased1 = TTbar_biased_700to1000_unweighted['c_eff_denominator_s01'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
c_eff_denominator_pt_s01_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_denominator_s01'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')

c_eff_numerator_pt_s01 = c_eff_numerator_pt_s01_biased1.copy()
c_eff_numerator_pt_s01.add(c_eff_numerator_pt_s01_biased2)
c_eff_denominator_pt_s01 = c_eff_denominator_pt_s01_biased1.copy()
c_eff_denominator_pt_s01.add(c_eff_denominator_pt_s01_biased2)

udsg_eff_numerator_pt_s01_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_numerator_s01'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_numerator_pt_s01_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_numerator_s01'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')
udsg_eff_denominator_pt_s01_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_denominator_s01'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_denominator_pt_s01_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_denominator_s01'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')

udsg_eff_numerator_pt_s01 = udsg_eff_numerator_pt_s01_biased1.copy()
udsg_eff_numerator_pt_s01.add(udsg_eff_numerator_pt_s01_biased2)
udsg_eff_denominator_pt_s01 = udsg_eff_denominator_pt_s01_biased1.copy()
udsg_eff_denominator_pt_s01.add(udsg_eff_denominator_pt_s01_biased2)

b_eff_pt_s01 = hist.plotratio(num = b_eff_numerator_pt_s01, denom = b_eff_denominator_pt_s01, 
                              ax=ax01, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                              unc = 'num')
c_eff_pt_s01 = hist.plotratio(num = c_eff_numerator_pt_s01, denom = c_eff_denominator_pt_s01, 
                              ax=ax01, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                              unc = 'num')
udsg_eff_pt_s01 = hist.plotratio(num = udsg_eff_numerator_pt_s01, denom = udsg_eff_denominator_pt_s01, 
                              ax=ax01, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                              unc = 'num')

ax01.set_title(r'SubJet$_{01}\left(p_T\right)$')
ax01.set_ylabel('flavor-Tag Efficiency')
ax01.set_ylim(bottom=0., top=1.)

# ------------------------------------------------------------------------------ #

b_eff_numerator_pt_s02_biased1 = TTbar_biased_700to1000_unweighted['b_eff_numerator_s02'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
b_eff_numerator_pt_s02_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_numerator_s02'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')
b_eff_denominator_pt_s02_biased1 = TTbar_biased_700to1000_unweighted['b_eff_denominator_s02'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
b_eff_denominator_pt_s02_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_denominator_s02'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')

b_eff_numerator_pt_s02 = b_eff_numerator_pt_s02_biased1.copy()
b_eff_numerator_pt_s02.add(b_eff_numerator_pt_s02_biased2)
b_eff_denominator_pt_s02 = b_eff_denominator_pt_s02_biased1.copy()
b_eff_denominator_pt_s02.add(b_eff_denominator_pt_s02_biased2)

c_eff_numerator_pt_s02_biased1 = TTbar_biased_700to1000_unweighted['c_eff_numerator_s02'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
c_eff_numerator_pt_s02_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_numerator_s02'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')
c_eff_denominator_pt_s02_biased1 = TTbar_biased_700to1000_unweighted['c_eff_denominator_s02'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
c_eff_denominator_pt_s02_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_denominator_s02'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')

c_eff_numerator_pt_s02 = c_eff_numerator_pt_s02_biased1.copy()
c_eff_numerator_pt_s02.add(c_eff_numerator_pt_s02_biased2)
c_eff_denominator_pt_s02 = c_eff_denominator_pt_s02_biased1.copy()
c_eff_denominator_pt_s02.add(c_eff_denominator_pt_s02_biased2)

udsg_eff_numerator_pt_s02_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_numerator_s02'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_numerator_pt_s02_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_numerator_s02'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')
udsg_eff_denominator_pt_s02_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_denominator_s02'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_denominator_pt_s02_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_denominator_s02'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')

udsg_eff_numerator_pt_s02 = udsg_eff_numerator_pt_s02_biased1.copy()
udsg_eff_numerator_pt_s02.add(udsg_eff_numerator_pt_s02_biased2)
udsg_eff_denominator_pt_s02 = udsg_eff_denominator_pt_s02_biased1.copy()
udsg_eff_denominator_pt_s02.add(udsg_eff_denominator_pt_s02_biased2)

b_eff_pt_s02 = hist.plotratio(num = b_eff_numerator_pt_s02, denom = b_eff_denominator_pt_s02, 
                              ax=ax02, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                              unc = 'num')
c_eff_pt_s02 = hist.plotratio(num = c_eff_numerator_pt_s02, denom = c_eff_denominator_pt_s02, 
                              ax=ax02, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                              unc = 'num')
udsg_eff_pt_s02 = hist.plotratio(num = udsg_eff_numerator_pt_s02, denom = udsg_eff_denominator_pt_s02, 
                              ax=ax02, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                              unc = 'num')

ax02.set_title(r'SubJet$_{02}\left(p_T\right)$')
ax02.set_ylabel('')
ax02.set_ylim(bottom=0., top=1.)

# ------------------------------------------------------------------------------ #

b_eff_numerator_pt_s11_biased1 = TTbar_biased_700to1000_unweighted['b_eff_numerator_s11'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
b_eff_numerator_pt_s11_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_numerator_s11'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')
b_eff_denominator_pt_s11_biased1 = TTbar_biased_700to1000_unweighted['b_eff_denominator_s11'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
b_eff_denominator_pt_s11_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_denominator_s11'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')

b_eff_numerator_pt_s11 = b_eff_numerator_pt_s11_biased1.copy()
b_eff_numerator_pt_s11.add(b_eff_numerator_pt_s11_biased2)
b_eff_denominator_pt_s11 = b_eff_denominator_pt_s11_biased1.copy()
b_eff_denominator_pt_s11.add(b_eff_denominator_pt_s11_biased2)

c_eff_numerator_pt_s11_biased1 = TTbar_biased_700to1000_unweighted['c_eff_numerator_s11'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
c_eff_numerator_pt_s11_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_numerator_s11'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')
c_eff_denominator_pt_s11_biased1 = TTbar_biased_700to1000_unweighted['c_eff_denominator_s11'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
c_eff_denominator_pt_s11_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_denominator_s11'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')

c_eff_numerator_pt_s11 = c_eff_numerator_pt_s11_biased1.copy()
c_eff_numerator_pt_s11.add(c_eff_numerator_pt_s11_biased2)
c_eff_denominator_pt_s11 = c_eff_denominator_pt_s11_biased1.copy()
c_eff_denominator_pt_s11.add(c_eff_denominator_pt_s11_biased2)

udsg_eff_numerator_pt_s11_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_numerator_s11'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_numerator_pt_s11_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_numerator_s11'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')
udsg_eff_denominator_pt_s11_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_denominator_s11'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_denominator_pt_s11_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_denominator_s11'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')

udsg_eff_numerator_pt_s11 = udsg_eff_numerator_pt_s11_biased1.copy()
udsg_eff_numerator_pt_s11.add(udsg_eff_numerator_pt_s11_biased2)
udsg_eff_denominator_pt_s11 = udsg_eff_denominator_pt_s11_biased1.copy()
udsg_eff_denominator_pt_s11.add(udsg_eff_denominator_pt_s11_biased2)

b_eff_pt_s11 = hist.plotratio(num = b_eff_numerator_pt_s11, denom = b_eff_denominator_pt_s11, 
                              ax=ax11, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                              unc = 'num')
c_eff_pt_s11 = hist.plotratio(num = c_eff_numerator_pt_s11, denom = c_eff_denominator_pt_s11, 
                              ax=ax11, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                              unc = 'num')
udsg_eff_pt_s11 = hist.plotratio(num = udsg_eff_numerator_pt_s11, denom = udsg_eff_denominator_pt_s11, 
                              ax=ax11, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                              unc = 'num')

ax11.set_title(r'SubJet$_{11}\left(p_T\right)$')
ax11.set_ylabel('')
ax11.set_ylim(bottom=0., top=1.)

# ------------------------------------------------------------------------------ #

b_eff_numerator_pt_s12_biased1 = TTbar_biased_700to1000_unweighted['b_eff_numerator_s12'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
b_eff_numerator_pt_s12_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_numerator_s12'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')
b_eff_denominator_pt_s12_biased1 = TTbar_biased_700to1000_unweighted['b_eff_denominator_s12'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
b_eff_denominator_pt_s12_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_denominator_s12'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')

b_eff_numerator_pt_s12 = b_eff_numerator_pt_s12_biased1.copy()
b_eff_numerator_pt_s12.add(b_eff_numerator_pt_s12_biased2)
b_eff_denominator_pt_s12 = b_eff_denominator_pt_s12_biased1.copy()
b_eff_denominator_pt_s12.add(b_eff_denominator_pt_s12_biased2)

c_eff_numerator_pt_s12_biased1 = TTbar_biased_700to1000_unweighted['c_eff_numerator_s12'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
c_eff_numerator_pt_s12_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_numerator_s12'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')
c_eff_denominator_pt_s12_biased1 = TTbar_biased_700to1000_unweighted['c_eff_denominator_s12'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
c_eff_denominator_pt_s12_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_denominator_s12'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')

c_eff_numerator_pt_s12 = c_eff_numerator_pt_s12_biased1.copy()
c_eff_numerator_pt_s12.add(c_eff_numerator_pt_s12_biased2)
c_eff_denominator_pt_s12 = c_eff_denominator_pt_s12_biased1.copy()
c_eff_denominator_pt_s12.add(c_eff_denominator_pt_s12_biased2)

udsg_eff_numerator_pt_s12_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_numerator_s12'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_numerator_pt_s12_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_numerator_s12'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')
udsg_eff_denominator_pt_s12_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_denominator_s12'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_denominator_pt_s12_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_denominator_s12'+binwidth].integrate('subjeteta').integrate('dataset', 'TTbar_biased_1000toInf')

udsg_eff_numerator_pt_s12 = udsg_eff_numerator_pt_s12_biased1.copy()
udsg_eff_numerator_pt_s12.add(udsg_eff_numerator_pt_s12_biased2)
udsg_eff_denominator_pt_s12 = udsg_eff_denominator_pt_s12_biased1.copy()
udsg_eff_denominator_pt_s12.add(udsg_eff_denominator_pt_s12_biased2)

b_eff_pt_s12 = hist.plotratio(num = b_eff_numerator_pt_s12, denom = b_eff_denominator_pt_s12, 
                              ax=ax12, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                              unc = 'num')
c_eff_pt_s12 = hist.plotratio(num = c_eff_numerator_pt_s12, denom = c_eff_denominator_pt_s12, 
                              ax=ax12, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                              unc = 'num')
udsg_eff_pt_s12 = hist.plotratio(num = udsg_eff_numerator_pt_s12, denom = udsg_eff_denominator_pt_s12, 
                              ax=ax12, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                              unc = 'num')

ax12.set_title(r'SubJet$_{12}\left(p_T\right)$')
ax12.set_ylabel('')
ax12.set_ylim(bottom=0., top=1.)
leg = ax11.legend(labels=[r'$b_{eff}$',r'$c_{eff}$',r'$udsg_{eff}$'], bbox_to_anchor=(0.7, 1.5), loc='upper right')

Dataset = 'TTbarBiasedSum'
fig.suptitle(Dataset + ' Sample')

filename = Dataset + '_flavor_efficiency_pt' + filename_bw + '.png'

In [ ]:
SaveDirectory = maindirectory + '/FlavorTagEfficiencies/images/1D/flavortags_pt/' # split histograms into subdirectories
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
if PlotWithLargeBins == True:
    binwidth = '_manualbins'
    filename_bw = '_large_bins'
else:
    binwidth = ''
    filename_bw = ''
for dataset,output in Datasets.items(): 
    plt.rcParams.update({
    'font.size': 14,
    'axes.titlesize': 18,
    'axes.labelsize': 18,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12
    })
    fig, (ax01, ax02, ax11, ax12) = plt.subplots(1, 4, figsize=(17,5), sharex=True)

    b_eff_numerator_pt_s01 = output['b_eff_numerator_s01'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    b_eff_denominator_pt_s01 = output['b_eff_denominator_s01'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    c_eff_numerator_pt_s01 = output['c_eff_numerator_s01'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    c_eff_denominator_pt_s01 = output['c_eff_denominator_s01'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    udsg_eff_numerator_pt_s01 = output['udsg_eff_numerator_s01'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    udsg_eff_denominator_pt_s01 = output['udsg_eff_denominator_s01'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    
    b_eff_pt_s01 = hist.plotratio(num = b_eff_numerator_pt_s01, denom = b_eff_denominator_pt_s01, 
                                  ax=ax01, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                                  unc = 'num')
    c_eff_pt_s01 = hist.plotratio(num = c_eff_numerator_pt_s01, denom = c_eff_denominator_pt_s01, 
                                  ax=ax01, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                                  unc = 'num')
    udsg_eff_pt_s01 = hist.plotratio(num = udsg_eff_numerator_pt_s01, denom = udsg_eff_denominator_pt_s01, 
                                  ax=ax01, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                                  unc = 'num')
    
    ax01.set_title(r'SubJet$_{01}\left(p_T\right)$')
    ax01.set_ylabel('flavor-Tag Efficiency')
    ax01.set_ylim(bottom=0., top=1.)

    # ------------------------------------------------------------------------------ #

    b_eff_numerator_pt_s02 = output['b_eff_numerator_s02'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    b_eff_denominator_pt_s02 = output['b_eff_denominator_s02'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    c_eff_numerator_pt_s02 = output['c_eff_numerator_s02'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    c_eff_denominator_pt_s02 = output['c_eff_denominator_s02'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    udsg_eff_numerator_pt_s02 = output['udsg_eff_numerator_s02'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    udsg_eff_denominator_pt_s02 = output['udsg_eff_denominator_s02'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    
    b_eff_pt_s02 = hist.plotratio(num = b_eff_numerator_pt_s02, denom = b_eff_denominator_pt_s02, 
                                  ax=ax02, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                                  unc = 'num')
    c_eff_pt_s02 = hist.plotratio(num = c_eff_numerator_pt_s02, denom = c_eff_denominator_pt_s02, 
                                  ax=ax02, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                                  unc = 'num')
    udsg_eff_pt_s02 = hist.plotratio(num = udsg_eff_numerator_pt_s02, denom = udsg_eff_denominator_pt_s02, 
                                  ax=ax02, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                                  unc = 'num')
    
    ax02.set_title(r'SubJet$_{02}\left(p_T\right)$')
    ax02.set_ylabel('')
    ax02.set_ylim(bottom=0., top=1.)

    # ------------------------------------------------------------------------------ #

    b_eff_numerator_pt_s11 = output['b_eff_numerator_s11'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    b_eff_denominator_pt_s11 = output['b_eff_denominator_s11'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    c_eff_numerator_pt_s11 = output['c_eff_numerator_s11'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    c_eff_denominator_pt_s11 = output['c_eff_denominator_s11'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    udsg_eff_numerator_pt_s11 = output['udsg_eff_numerator_s11'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    udsg_eff_denominator_pt_s11 = output['udsg_eff_denominator_s11'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    
    b_eff_pt_s11 = hist.plotratio(num = b_eff_numerator_pt_s11, denom = b_eff_denominator_pt_s11, 
                                  ax=ax11, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                                  unc = 'num')
    c_eff_pt_s11 = hist.plotratio(num = c_eff_numerator_pt_s11, denom = c_eff_denominator_pt_s11, 
                                  ax=ax11, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                                  unc = 'num')
    udsg_eff_pt_s11 = hist.plotratio(num = udsg_eff_numerator_pt_s11, denom = udsg_eff_denominator_pt_s11, 
                                  ax=ax11, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                                  unc = 'num')
    
    ax11.set_title(r'SubJet$_{11}\left(p_T\right)$')
    ax11.set_ylabel('')
    ax11.set_ylim(bottom=0., top=1.)

    # ------------------------------------------------------------------------------ #

    b_eff_numerator_pt_s12 = output['b_eff_numerator_s12'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    b_eff_denominator_pt_s12 = output['b_eff_denominator_s12'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    c_eff_numerator_pt_s12 = output['c_eff_numerator_s12'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    c_eff_denominator_pt_s12 = output['c_eff_denominator_s12'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    udsg_eff_numerator_pt_s12 = output['udsg_eff_numerator_s12'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    udsg_eff_denominator_pt_s12 = output['udsg_eff_denominator_s12'+binwidth].integrate('subjeteta').integrate('dataset', dataset)
    
    b_eff_pt_s12 = hist.plotratio(num = b_eff_numerator_pt_s12, denom = b_eff_denominator_pt_s12, 
                                  ax=ax12, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                                  unc = 'num')
    c_eff_pt_s12 = hist.plotratio(num = c_eff_numerator_pt_s12, denom = c_eff_denominator_pt_s12, 
                                  ax=ax12, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                                  unc = 'num')
    udsg_eff_pt_s12 = hist.plotratio(num = udsg_eff_numerator_pt_s12, denom = udsg_eff_denominator_pt_s12, 
                                  ax=ax12, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                                  unc = 'num')
    
    ax12.set_title(r'SubJet$_{12}\left(p_T\right)$')
    ax12.set_ylabel('')
    ax12.set_ylim(bottom=0., top=1.)
    leg = ax11.legend(labels=[r'$b_{eff}$',r'$c_{eff}$',r'$udsg_{eff}$'], bbox_to_anchor=(0.7, 1.4), loc='upper right')
    fig.suptitle(dataset + ' Sample')

    filename = dataset + '_flavor_efficiency_pt' + filename_bw + '.png'
    print(filename + ' generated')
    plt.savefig(SaveDirectory+filename, bbox_inches="tight")

# Plot (and save) the flavor tagging efficiencies as functions of $\eta$ 

In [ ]:
SaveDirectory = maindirectory + '/FlavorTagEfficiencies/images/1D/flavortags_eta/' # split histograms into subdirectories
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
if PlotWithLargeBins == True:
    binwidth = '_manualbins'
    filename_bw = '_large_bins'
else:
    binwidth = ''
    filename_bw = ''

plt.rcParams.update({
'font.size': 14,
'axes.titlesize': 18,
'axes.labelsize': 18,
'xtick.labelsize': 12,
'ytick.labelsize': 12
})
fig, (ax01, ax02, ax11, ax12) = plt.subplots(1, 4, figsize=(17,5), sharex=True)

b_eff_numerator_eta_s01_biased1 = TTbar_biased_700to1000_unweighted['b_eff_numerator_s01'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
b_eff_numerator_eta_s01_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_numerator_s01'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')
b_eff_denominator_eta_s01_biased1 = TTbar_biased_700to1000_unweighted['b_eff_denominator_s01'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
b_eff_denominator_eta_s01_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_denominator_s01'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')

b_eff_numerator_eta_s01 = b_eff_numerator_eta_s01_biased1.copy()
b_eff_numerator_eta_s01.add(b_eff_numerator_eta_s01_biased2)
b_eff_denominator_eta_s01 = b_eff_denominator_eta_s01_biased1.copy()
b_eff_denominator_eta_s01.add(b_eff_denominator_eta_s01_biased2)

c_eff_numerator_eta_s01_biased1 = TTbar_biased_700to1000_unweighted['c_eff_numerator_s01'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
c_eff_numerator_eta_s01_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_numerator_s01'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')
c_eff_denominator_eta_s01_biased1 = TTbar_biased_700to1000_unweighted['c_eff_denominator_s01'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
c_eff_denominator_eta_s01_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_denominator_s01'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')

c_eff_numerator_eta_s01 = c_eff_numerator_eta_s01_biased1.copy()
c_eff_numerator_eta_s01.add(c_eff_numerator_eta_s01_biased2)
c_eff_denominator_eta_s01 = c_eff_denominator_eta_s01_biased1.copy()
c_eff_denominator_eta_s01.add(c_eff_denominator_eta_s01_biased2)

udsg_eff_numerator_eta_s01_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_numerator_s01'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_numerator_eta_s01_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_numerator_s01'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')
udsg_eff_denominator_eta_s01_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_denominator_s01'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_denominator_eta_s01_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_denominator_s01'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')

udsg_eff_numerator_eta_s01 = udsg_eff_numerator_eta_s01_biased1.copy()
udsg_eff_numerator_eta_s01.add(udsg_eff_numerator_eta_s01_biased2)
udsg_eff_denominator_eta_s01 = udsg_eff_denominator_eta_s01_biased1.copy()
udsg_eff_denominator_eta_s01.add(udsg_eff_denominator_eta_s01_biased2)

b_eff_eta_s01 = hist.plotratio(num = b_eff_numerator_eta_s01, denom = b_eff_denominator_eta_s01, 
                              ax=ax01, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                              unc = 'num')
c_eff_eta_s01 = hist.plotratio(num = c_eff_numerator_eta_s01, denom = c_eff_denominator_eta_s01, 
                              ax=ax01, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                              unc = 'num')
udsg_eff_eta_s01 = hist.plotratio(num = udsg_eff_numerator_eta_s01, denom = udsg_eff_denominator_eta_s01, 
                              ax=ax01, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                              unc = 'num')

ax01.set_title(r'SubJet$_{01}\left(\eta\right)$')
ax01.set_ylabel('flavor-Tag Efficiency')
ax01.set_ylim(bottom=0., top=1.)

# ------------------------------------------------------------------------------ #

b_eff_numerator_eta_s02_biased1 = TTbar_biased_700to1000_unweighted['b_eff_numerator_s02'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
b_eff_numerator_eta_s02_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_numerator_s02'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')
b_eff_denominator_eta_s02_biased1 = TTbar_biased_700to1000_unweighted['b_eff_denominator_s02'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
b_eff_denominator_eta_s02_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_denominator_s02'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')

b_eff_numerator_eta_s02 = b_eff_numerator_eta_s02_biased1.copy()
b_eff_numerator_eta_s02.add(b_eff_numerator_eta_s02_biased2)
b_eff_denominator_eta_s02 = b_eff_denominator_eta_s02_biased1.copy()
b_eff_denominator_eta_s02.add(b_eff_denominator_eta_s02_biased2)

c_eff_numerator_eta_s02_biased1 = TTbar_biased_700to1000_unweighted['c_eff_numerator_s02'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
c_eff_numerator_eta_s02_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_numerator_s02'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')
c_eff_denominator_eta_s02_biased1 = TTbar_biased_700to1000_unweighted['c_eff_denominator_s02'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
c_eff_denominator_eta_s02_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_denominator_s02'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')

c_eff_numerator_eta_s02 = c_eff_numerator_eta_s02_biased1.copy()
c_eff_numerator_eta_s02.add(c_eff_numerator_eta_s02_biased2)
c_eff_denominator_eta_s02 = c_eff_denominator_eta_s02_biased1.copy()
c_eff_denominator_eta_s02.add(c_eff_denominator_eta_s02_biased2)

udsg_eff_numerator_eta_s02_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_numerator_s02'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_numerator_eta_s02_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_numerator_s02'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')
udsg_eff_denominator_eta_s02_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_denominator_s02'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_denominator_eta_s02_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_denominator_s02'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')

udsg_eff_numerator_eta_s02 = udsg_eff_numerator_eta_s02_biased1.copy()
udsg_eff_numerator_eta_s02.add(udsg_eff_numerator_eta_s02_biased2)
udsg_eff_denominator_eta_s02 = udsg_eff_denominator_eta_s02_biased1.copy()
udsg_eff_denominator_eta_s02.add(udsg_eff_denominator_eta_s02_biased2)

b_eff_eta_s02 = hist.plotratio(num = b_eff_numerator_eta_s02, denom = b_eff_denominator_eta_s02, 
                              ax=ax02, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                              unc = 'num')
c_eff_eta_s02 = hist.plotratio(num = c_eff_numerator_eta_s02, denom = c_eff_denominator_eta_s02, 
                              ax=ax02, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                              unc = 'num')
udsg_eff_eta_s02 = hist.plotratio(num = udsg_eff_numerator_eta_s02, denom = udsg_eff_denominator_eta_s02, 
                              ax=ax02, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                              unc = 'num')

ax02.set_title(r'SubJet$_{02}\left(\eta\right)$')
ax02.set_ylabel('')
ax02.set_ylim(bottom=0., top=1.)

# ------------------------------------------------------------------------------ #

b_eff_numerator_eta_s11_biased1 = TTbar_biased_700to1000_unweighted['b_eff_numerator_s11'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
b_eff_numerator_eta_s11_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_numerator_s11'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')
b_eff_denominator_eta_s11_biased1 = TTbar_biased_700to1000_unweighted['b_eff_denominator_s11'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
b_eff_denominator_eta_s11_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_denominator_s11'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')

b_eff_numerator_eta_s11 = b_eff_numerator_eta_s11_biased1.copy()
b_eff_numerator_eta_s11.add(b_eff_numerator_eta_s11_biased2)
b_eff_denominator_eta_s11 = b_eff_denominator_eta_s11_biased1.copy()
b_eff_denominator_eta_s11.add(b_eff_denominator_eta_s11_biased2)

c_eff_numerator_eta_s11_biased1 = TTbar_biased_700to1000_unweighted['c_eff_numerator_s11'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
c_eff_numerator_eta_s11_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_numerator_s11'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')
c_eff_denominator_eta_s11_biased1 = TTbar_biased_700to1000_unweighted['c_eff_denominator_s11'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
c_eff_denominator_eta_s11_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_denominator_s11'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')

c_eff_numerator_eta_s11 = c_eff_numerator_eta_s11_biased1.copy()
c_eff_numerator_eta_s11.add(c_eff_numerator_eta_s11_biased2)
c_eff_denominator_eta_s11 = c_eff_denominator_eta_s11_biased1.copy()
c_eff_denominator_eta_s11.add(c_eff_denominator_eta_s11_biased2)

udsg_eff_numerator_eta_s11_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_numerator_s11'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_numerator_eta_s11_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_numerator_s11'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')
udsg_eff_denominator_eta_s11_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_denominator_s11'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_denominator_eta_s11_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_denominator_s11'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')

udsg_eff_numerator_eta_s11 = udsg_eff_numerator_eta_s11_biased1.copy()
udsg_eff_numerator_eta_s11.add(udsg_eff_numerator_eta_s11_biased2)
udsg_eff_denominator_eta_s11 = udsg_eff_denominator_eta_s11_biased1.copy()
udsg_eff_denominator_eta_s11.add(udsg_eff_denominator_eta_s11_biased2)

b_eff_eta_s11 = hist.plotratio(num = b_eff_numerator_eta_s11, denom = b_eff_denominator_eta_s11, 
                              ax=ax11, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                              unc = 'num')
c_eff_eta_s11 = hist.plotratio(num = c_eff_numerator_eta_s11, denom = c_eff_denominator_eta_s11, 
                              ax=ax11, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                              unc = 'num')
udsg_eff_eta_s11 = hist.plotratio(num = udsg_eff_numerator_eta_s11, denom = udsg_eff_denominator_eta_s11, 
                              ax=ax11, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                              unc = 'num')

ax11.set_title(r'SubJet$_{11}\left(\eta\right)$')
ax11.set_ylabel('')
ax11.set_ylim(bottom=0., top=1.)

# ------------------------------------------------------------------------------ #

b_eff_numerator_eta_s12_biased1 = TTbar_biased_700to1000_unweighted['b_eff_numerator_s12'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
b_eff_numerator_eta_s12_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_numerator_s12'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')
b_eff_denominator_eta_s12_biased1 = TTbar_biased_700to1000_unweighted['b_eff_denominator_s12'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
b_eff_denominator_eta_s12_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_denominator_s12'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')

b_eff_numerator_eta_s12 = b_eff_numerator_eta_s12_biased1.copy()
b_eff_numerator_eta_s12.add(b_eff_numerator_eta_s12_biased2)
b_eff_denominator_eta_s12 = b_eff_denominator_eta_s12_biased1.copy()
b_eff_denominator_eta_s12.add(b_eff_denominator_eta_s12_biased2)

c_eff_numerator_eta_s12_biased1 = TTbar_biased_700to1000_unweighted['c_eff_numerator_s12'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
c_eff_numerator_eta_s12_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_numerator_s12'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')
c_eff_denominator_eta_s12_biased1 = TTbar_biased_700to1000_unweighted['c_eff_denominator_s12'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
c_eff_denominator_eta_s12_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_denominator_s12'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')

c_eff_numerator_eta_s12 = c_eff_numerator_eta_s12_biased1.copy()
c_eff_numerator_eta_s12.add(c_eff_numerator_eta_s12_biased2)
c_eff_denominator_eta_s12 = c_eff_denominator_eta_s12_biased1.copy()
c_eff_denominator_eta_s12.add(c_eff_denominator_eta_s12_biased2)

udsg_eff_numerator_eta_s12_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_numerator_s12'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_numerator_eta_s12_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_numerator_s12'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')
udsg_eff_denominator_eta_s12_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_denominator_s12'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_denominator_eta_s12_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_denominator_s12'+binwidth].integrate('subjetpt').integrate('dataset', 'TTbar_biased_1000toInf')

udsg_eff_numerator_eta_s12 = udsg_eff_numerator_eta_s12_biased1.copy()
udsg_eff_numerator_eta_s12.add(udsg_eff_numerator_eta_s12_biased2)
udsg_eff_denominator_eta_s12 = udsg_eff_denominator_eta_s12_biased1.copy()
udsg_eff_denominator_eta_s12.add(udsg_eff_denominator_eta_s12_biased2)

b_eff_eta_s12 = hist.plotratio(num = b_eff_numerator_eta_s12, denom = b_eff_denominator_eta_s12, 
                              ax=ax12, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                              unc = 'num')
c_eff_eta_s12 = hist.plotratio(num = c_eff_numerator_eta_s12, denom = c_eff_denominator_eta_s12, 
                              ax=ax12, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                              unc = 'num')
udsg_eff_eta_s12 = hist.plotratio(num = udsg_eff_numerator_eta_s12, denom = udsg_eff_denominator_eta_s12, 
                              ax=ax12, clear=False,
                              error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                              unc = 'num')

ax12.set_title(r'SubJet$_{12}\left(\eta\right)$')
ax12.set_ylabel('')
ax12.set_ylim(bottom=0., top=1.)
leg = ax11.legend(labels=[r'$b_{eff}$',r'$c_{eff}$',r'$udsg_{eff}$'], bbox_to_anchor=(0.7, 1.5), loc='upper right')

Dataset = 'TTbarBiasedSum'
fig.suptitle(Dataset + ' Sample')

filename = Dataset + '_flavor_efficiency_eta' + filename_bw + '.png'

In [ ]:
SaveDirectory = maindirectory + '/FlavorTagEfficiencies/images/1D/flavortags_eta/' # split histograms into subdirectories
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
if PlotWithLargeBins == True:
    binwidth = '_manualbins'
    filename_bw = '_large_bins'
else:
    binwidth = ''
    filename_bw = ''
for dataset,output in Datasets.items(): 
    plt.rcParams.update({
    'font.size': 14,
    'axes.titlesize': 18,
    'axes.labelsize': 18,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12
    })
    fig, (ax01, ax02, ax11, ax12) = plt.subplots(1, 4, figsize=(17,5), sharex=True)

    b_eff_numerator_eta_s01 = output['b_eff_numerator_s01'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    b_eff_denominator_eta_s01 = output['b_eff_denominator_s01'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    c_eff_numerator_eta_s01 = output['c_eff_numerator_s01'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    c_eff_denominator_eta_s01 = output['c_eff_denominator_s01'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    udsg_eff_numerator_eta_s01 = output['udsg_eff_numerator_s01'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    udsg_eff_denominator_eta_s01 = output['udsg_eff_denominator_s01'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    
    b_eff_pt_s01 = hist.plotratio(num = b_eff_numerator_eta_s01, denom = b_eff_denominator_eta_s01, 
                                  ax=ax01, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                                  unc = 'num')
    c_eff_pt_s01 = hist.plotratio(num = c_eff_numerator_eta_s01, denom = c_eff_denominator_eta_s01, 
                                  ax=ax01, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                                  unc = 'num')
    udsg_eff_pt_s01 = hist.plotratio(num = udsg_eff_numerator_eta_s01, denom = udsg_eff_denominator_eta_s01, 
                                  ax=ax01, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                                  unc = 'num')
    
    ax01.set_title(r'SubJet$_{01}\left(\eta\right)$')
    ax01.set_ylabel('flavor-Tag Efficiency')
    ax01.set_ylim(bottom=0., top=1.)

    # ------------------------------------------------------------------------------ #

    b_eff_numerator_eta_s02 = output['b_eff_numerator_s02'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    b_eff_denominator_eta_s02 = output['b_eff_denominator_s02'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    c_eff_numerator_eta_s02 = output['c_eff_numerator_s02'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    c_eff_denominator_eta_s02 = output['c_eff_denominator_s02'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    udsg_eff_numerator_eta_s02 = output['udsg_eff_numerator_s02'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    udsg_eff_denominator_eta_s02 = output['udsg_eff_denominator_s02'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    
    b_eff_pt_s02 = hist.plotratio(num = b_eff_numerator_eta_s02, denom = b_eff_denominator_eta_s02, 
                                  ax=ax02, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                                  unc = 'num')
    c_eff_pt_s02 = hist.plotratio(num = c_eff_numerator_eta_s02, denom = c_eff_denominator_eta_s02, 
                                  ax=ax02, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                                  unc = 'num')
    udsg_eff_pt_s02 = hist.plotratio(num = udsg_eff_numerator_eta_s02, denom = udsg_eff_denominator_eta_s02, 
                                  ax=ax02, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                                  unc = 'num')
    
    ax02.set_title(r'SubJet$_{02}\left(\eta\right)$')
    ax02.set_ylabel('')
    ax02.set_ylim(bottom=0., top=1.)

    # ------------------------------------------------------------------------------ #

    b_eff_numerator_eta_s11 = output['b_eff_numerator_s11'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    b_eff_denominator_eta_s11 = output['b_eff_denominator_s11'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    c_eff_numerator_eta_s11 = output['c_eff_numerator_s11'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    c_eff_denominator_eta_s11 = output['c_eff_denominator_s11'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    udsg_eff_numerator_eta_s11 = output['udsg_eff_numerator_s11'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    udsg_eff_denominator_eta_s11 = output['udsg_eff_denominator_s11'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    
    b_eff_pt_s11 = hist.plotratio(num = b_eff_numerator_eta_s11, denom = b_eff_denominator_eta_s11, 
                                  ax=ax11, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                                  unc = 'num')
    c_eff_pt_s11 = hist.plotratio(num = c_eff_numerator_eta_s11, denom = c_eff_denominator_eta_s11, 
                                  ax=ax11, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                                  unc = 'num')
    udsg_eff_pt_s11 = hist.plotratio(num = udsg_eff_numerator_eta_s11, denom = udsg_eff_denominator_eta_s11, 
                                  ax=ax11, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                                  unc = 'num')
    
    ax11.set_title(r'SubJet$_{11}\left(\eta\right)$')
    ax11.set_ylabel('')
    ax11.set_ylim(bottom=0., top=1.)

    # ------------------------------------------------------------------------------ #

    b_eff_numerator_eta_s12 = output['b_eff_numerator_s12'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    b_eff_denominator_eta_s12 = output['b_eff_denominator_s12'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    c_eff_numerator_eta_s12 = output['c_eff_numerator_s12'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    c_eff_denominator_eta_s12 = output['c_eff_denominator_s12'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    udsg_eff_numerator_eta_s12 = output['udsg_eff_numerator_s12'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    udsg_eff_denominator_eta_s12 = output['udsg_eff_denominator_s12'+binwidth].integrate('subjetpt').integrate('dataset', dataset)
    
    b_eff_pt_s12 = hist.plotratio(num = b_eff_numerator_eta_s12, denom = b_eff_denominator_eta_s12, 
                                  ax=ax12, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                                  unc = 'num')
    c_eff_pt_s12 = hist.plotratio(num = c_eff_numerator_eta_s12, denom = c_eff_denominator_eta_s12, 
                                  ax=ax12, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                                  unc = 'num')
    udsg_eff_pt_s12 = hist.plotratio(num = udsg_eff_numerator_eta_s12, denom = udsg_eff_denominator_eta_s12, 
                                  ax=ax12, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                                  unc = 'num')
    
    ax12.set_title(r'SubJet$_{12}\left(\eta\right)$')
    ax12.set_ylabel('')
    ax12.set_ylim(bottom=0., top=1.)
    leg = ax11.legend(labels=[r'$b_{eff}$',r'$c_{eff}$',r'$udsg_{eff}$'], bbox_to_anchor=(0.7, 1.4), loc='upper right')
    fig.suptitle(dataset + ' Sample')

    filename = dataset + '_flavor_efficiency_eta' + filename_bw + '.png'
    print(filename + ' generated')
    plt.savefig(SaveDirectory+filename, bbox_inches="tight")

# Define function to plot a 2D ratio histogram given two coffea.hist objects (the numerator and denominator). #
# Now, we will be able to see the efficiencies as functions of both $p_T$ and $\eta$.

In [ ]:
import mplhep as hep
from hist.intervals import ratio_uncertainty

In [ ]:
def plotratio2d(numerator, denominator, ax=None, cmap='Blues', cbar=True, norm_min=0., norm_max=1., alpha=None):
    NumeratorAxes = numerator.axes()
    DenominatorAxes = denominator.axes()
    
    # integer number of bins in this axis #
    NumeratorAxis1_BinNumber = NumeratorAxes[0].size - 3 # Subtract 3 to remove overflow
    NumeratorAxis2_BinNumber = NumeratorAxes[1].size - 3
    
    DenominatorAxis1_BinNumber = DenominatorAxes[0].size - 3 
    DenominatorAxis2_BinNumber = DenominatorAxes[1].size - 3 
    
    if(NumeratorAxis1_BinNumber != DenominatorAxis1_BinNumber 
       or NumeratorAxis2_BinNumber != DenominatorAxis2_BinNumber):
        raise Exception('Numerator and Denominator axes are different sizes; Cannot perform division.')
    else:
        Numerator = numerator.to_hist()
        Denominator = denominator.to_hist()

        ratio = Numerator / Denominator.values()

#         err_up, err_down = ratio_uncertainty(Numerator.values(), Denominator_fixed, 'poisson-ratio')

#         labels = []
#         for ra, u, d in zip(ratio.values().ravel(), err_up.ravel(), err_down.ravel()):
#             ra, u, d = f'{ra:.2f}', f'{u:.2f}', f'{d:.2f}'
#             st = '$'+ra+'_{-'+d+'}^{+'+u+'}$'
#             labels.append(st)
#         labels = np.array(labels).reshape(NumeratorAxis1_BinNumber,NumeratorAxis2_BinNumber)
        
        return hep.hist2dplot(ratio, ax=ax, cmap=cmap, norm=colors.Normalize(norm_min,norm_max), cbar=cbar, alpha=alpha)

# Plot b-tag $\epsilon\left(p_T,\ \eta\right)$
# Can either plot with small bins $(50\times50)$, large bins $(10\times10)$, or manual bins

In [ ]:
SaveDirectory = maindirectory + '/FlavorTagEfficiencies/images/2D/btags/' # split histograms into subdirectories
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
if PlotWithLargeBins == True:
    binwidth = '_manualbins' # selection of correct name of the Hist object
    filename_bw = '_large_bins' # choose a desired extension name to the filename for future clarity
else:
    binwidth = ''
    filename_bw = ''

plt.rcParams.update({
'font.size': 14,
'axes.titlesize': 18,
'axes.labelsize': 18,
'xtick.labelsize': 12,
'ytick.labelsize': 12
})
fig, (ax01, ax02, ax11, ax12) = plt.subplots(1, 4, figsize=(15,5), sharex=True)

b_eff_numerator_s01_biased1 = TTbar_biased_700to1000_unweighted['b_eff_numerator_s01'+binwidth].integrate('dataset', 'TTbar_biased_700to1000')
b_eff_numerator_s01_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_numerator_s01'+binwidth].integrate('dataset', 'TTbar_biased_1000toInf')
b_eff_denominator_s01_biased1 = TTbar_biased_700to1000_unweighted['b_eff_denominator_s01'+binwidth].integrate('dataset', 'TTbar_biased_700to1000')
b_eff_denominator_s01_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_denominator_s01'+binwidth].integrate('dataset', 'TTbar_biased_1000toInf')

b_eff_numerator_s01 = b_eff_numerator_s01_biased1.copy()
b_eff_numerator_s01.add(b_eff_numerator_s01_biased2)
b_eff_denominator_s01 = b_eff_denominator_s01_biased1.copy()
b_eff_denominator_s01.add(b_eff_denominator_s01_biased2)

b_eff_s01 = plotratio2d(b_eff_numerator_s01, b_eff_denominator_s01, ax=ax01, cbar=False)

ax01.set_title(r'b-tag $\epsilon_{01}\left(p_T,\ \eta\right)$')

#----------------------------------------------------------------------------------------#

b_eff_numerator_s02_biased1 = TTbar_biased_700to1000_unweighted['b_eff_numerator_s02'+binwidth].integrate('dataset', 'TTbar_biased_700to1000')
b_eff_numerator_s02_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_numerator_s02'+binwidth].integrate('dataset', 'TTbar_biased_1000toInf')
b_eff_denominator_s02_biased1 = TTbar_biased_700to1000_unweighted['b_eff_denominator_s02'+binwidth].integrate('dataset', 'TTbar_biased_700to1000')
b_eff_denominator_s02_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_denominator_s02'+binwidth].integrate('dataset', 'TTbar_biased_1000toInf')

b_eff_numerator_s02 = b_eff_numerator_s02_biased1.copy()
b_eff_numerator_s02.add(b_eff_numerator_s02_biased2)
b_eff_denominator_s02 = b_eff_denominator_s02_biased1.copy()
b_eff_denominator_s02.add(b_eff_denominator_s02_biased2)

b_eff_s02 = plotratio2d(b_eff_numerator_s02, b_eff_denominator_s02, ax=ax02, cbar=False)

ax02.set_title(r'b-tag $\epsilon_{02}\left(p_T,\ \eta\right)$')
ax02.set_ylabel('')
ax02.set_yticks([])

# #----------------------------------------------------------------------------------------#

b_eff_numerator_s11_biased1 = TTbar_biased_700to1000_unweighted['b_eff_numerator_s11'+binwidth].integrate('dataset', 'TTbar_biased_700to1000')
b_eff_numerator_s11_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_numerator_s11'+binwidth].integrate('dataset', 'TTbar_biased_1000toInf')
b_eff_denominator_s11_biased1 = TTbar_biased_700to1000_unweighted['b_eff_denominator_s11'+binwidth].integrate('dataset', 'TTbar_biased_700to1000')
b_eff_denominator_s11_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_denominator_s11'+binwidth].integrate('dataset', 'TTbar_biased_1000toInf')

b_eff_numerator_s11 = b_eff_numerator_s11_biased1.copy()
b_eff_numerator_s11.add(b_eff_numerator_s11_biased2)
b_eff_denominator_s11 = b_eff_denominator_s11_biased1.copy()
b_eff_denominator_s11.add(b_eff_denominator_s11_biased2)

b_eff_s11 = plotratio2d(b_eff_numerator_s11, b_eff_denominator_s11, ax=ax11, cbar=False)

ax11.set_title(r'b-tag $\epsilon_{11}\left(p_T,\ \eta\right)$')
ax11.set_ylabel('')
ax11.set_yticks([])

# #----------------------------------------------------------------------------------------#

b_eff_numerator_s12_biased1 = TTbar_biased_700to1000_unweighted['b_eff_numerator_s12'+binwidth].integrate('dataset', 'TTbar_biased_700to1000')
b_eff_numerator_s12_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_numerator_s12'+binwidth].integrate('dataset', 'TTbar_biased_1000toInf')
b_eff_denominator_s12_biased1 = TTbar_biased_700to1000_unweighted['b_eff_denominator_s12'+binwidth].integrate('dataset', 'TTbar_biased_700to1000')
b_eff_denominator_s12_biased2 = TTbar_biased_1000toInf_unweighted['b_eff_denominator_s12'+binwidth].integrate('dataset', 'TTbar_biased_1000toInf')

b_eff_numerator_s12 = b_eff_numerator_s12_biased1.copy()
b_eff_numerator_s12.add(b_eff_numerator_s12_biased2)
b_eff_denominator_s12 = b_eff_denominator_s12_biased1.copy()
b_eff_denominator_s12.add(b_eff_denominator_s12_biased2)

b_eff_s12 = plotratio2d(b_eff_numerator_s12, b_eff_denominator_s12, ax=ax12)#, cmap='plasma')

ax12.set_title(r'b-tag $\epsilon_{12}\left(p_T,\ \eta\right)$')
ax12.set_ylabel('')
ax12.set_yticks([])
ax12.set_xlim(0.,2000.)

# #----------------------------------------------------------------------------------------#
Dataset = 'TTbarBiasedSum'
fig.suptitle(Dataset + ' Sample')

filename = Dataset + '_btag_efficiency' + filename_bw + '.png'

In [ ]:
SaveDirectory = maindirectory + '/FlavorTagEfficiencies/images/2D/btags/' # split histograms into subdirectories
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
if PlotWithLargeBins == True:
    binwidth = '_manualbins'
    filename_bw = '_large_bins'
else:
    binwidth = ''
    filename_bw = ''
for dataset,output in Datasets.items():
    plt.rcParams.update({
    'font.size': 14,
    'axes.titlesize': 18,
    'axes.labelsize': 18,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12
    })
    fig, (ax01, ax02, ax11, ax12) = plt.subplots(1, 4, figsize=(15,5), sharex=True)

    b_eff_numerator_s01 = output['b_eff_numerator_s01'+binwidth].integrate('dataset', dataset)
    b_eff_denominator_s01 = output['b_eff_denominator_s01'+binwidth].integrate('dataset', dataset)

    b_eff_s01 = plotratio2d(b_eff_numerator_s01, b_eff_denominator_s01, ax=ax01, cbar=False)

    ax01.set_title(r'b-tag $\epsilon_{01}\left(p_T,\ \eta\right)$')
    if binwidth == '_manualbins':
        ax01.set_yticks([-2.4, -1.8, -1.2, -0.6, 0, 0.6, 1.2, 1.8, 2.4])
        ax01.set_yticklabels(['-2.4', '-1.8', '-1.2', '-0.6', '0', '0.6', '1.2', '1.8', '2.4'])
        ax01.set_xticks([0, 200, 400, 800, 1600, 3200])
        ax01.set_xticklabels(['', '.2', '.4', '.8', '1.6', '3.2'])
        ax01.set_xlabel(r'$Subjet\ p_T\ [TeV]$')
        ax01.grid()

    #----------------------------------------------------------------------------------------#

    b_eff_numerator_s02 = output['b_eff_numerator_s02'+binwidth].integrate('dataset', dataset)
    b_eff_denominator_s02 = output['b_eff_denominator_s02'+binwidth].integrate('dataset', dataset)

    b_eff_s02 = plotratio2d(b_eff_numerator_s02, b_eff_denominator_s02, ax=ax02, cbar=False)

    ax02.set_title(r'b-tag $\epsilon_{02}\left(p_T,\ \eta\right)$')
    ax02.set_ylabel('')
    if binwidth == '_manualbins':
        ax02.set_yticks([-2.4, -1.8, -1.2, -0.6, 0, 0.6, 1.2, 1.8, 2.4])
        ax02.set_yticklabels(['-2.4', '-1.8', '-1.2', '-0.6', '0', '0.6', '1.2', '1.8', '2.4'])
        ax02.set_xticks([0, 200, 400, 800, 1600, 3200])
        ax02.set_xticklabels(['', '.2', '.4', '.8', '1.6', '3.2'])
        ax02.set_xlabel(r'$Subjet\ p_T\ [TeV]$')
        ax02.grid()

    #----------------------------------------------------------------------------------------#

    b_eff_numerator_s11 = output['b_eff_numerator_s11'+binwidth].integrate('dataset', dataset)
    b_eff_denominator_s11 = output['b_eff_denominator_s11'+binwidth].integrate('dataset', dataset)

    b_eff_s11 = plotratio2d(b_eff_numerator_s11, b_eff_denominator_s11, ax=ax11, cbar=False)

    ax11.set_title(r'b-tag $\epsilon_{11}\left(p_T,\ \eta\right)$')
    ax11.set_ylabel('')
    if binwidth == '_manualbins':
        ax11.set_yticks([-2.4, -1.8, -1.2, -0.6, 0, 0.6, 1.2, 1.8, 2.4])
        ax11.set_yticklabels(['-2.4', '-1.8', '-1.2', '-0.6', '0', '0.6', '1.2', '1.8', '2.4'])
        ax11.set_xticks([0, 200, 400, 800, 1600, 3200])
        ax11.set_xticklabels(['', '.2', '.4', '.8', '1.6', '3.2'])
        ax11.set_xlabel(r'$Subjet\ p_T\ [TeV]$')
        ax11.grid()

    #----------------------------------------------------------------------------------------#

    b_eff_numerator_s12 = output['b_eff_numerator_s12'+binwidth].integrate('dataset', dataset)
    b_eff_denominator_s12 = output['b_eff_denominator_s12'+binwidth].integrate('dataset', dataset)

    b_eff_s12 = plotratio2d(b_eff_numerator_s12, b_eff_denominator_s12, ax=ax12)#, cmap='plasma')

    ax12.set_title(r'b-tag $\epsilon_{12}\left(p_T,\ \eta\right)$')
    ax12.set_ylabel('')
    if binwidth == '_manualbins':
        ax12.set_yticks([-2.4, -1.8, -1.2, -0.6, 0, 0.6, 1.2, 1.8, 2.4])
        ax12.set_yticklabels(['-2.4', '-1.8', '-1.2', '-0.6', '0', '0.6', '1.2', '1.8', '2.4'])
        ax12.set_xticks([0, 200, 400, 800, 1600, 3200])
        ax12.set_xticklabels(['', '.2', '.4', '.8', '1.6', '3.2'])
        ax12.set_xlabel(r'$Subjet\ p_T\ [TeV]$')
        ax12.grid()

    #----------------------------------------------------------------------------------------#
    fig.suptitle(dataset + ' Sample')

    filename = dataset + '_btag_efficiency' + filename_bw + '.png'
    print(filename + ' generated')
    plt.savefig(SaveDirectory+filename, bbox_inches="tight")

# Plot c-tag $\epsilon\left(p_T,\ \eta\right)$
# Can either plot with small bins $(50\times50)$ or large bins $(10\times10)$

In [ ]:
SaveDirectory = maindirectory + '/FlavorTagEfficiencies/images/2D/ctags/' # split histograms into subdirectories
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
if PlotWithLargeBins == True:
    binwidth = '_manualbins'
    filename_bw = '_large_bins'
else:
    binwidth = ''
    filename_bw = ''

plt.rcParams.update({
'font.size': 14,
'axes.titlesize': 18,
'axes.labelsize': 18,
'xtick.labelsize': 12,
'ytick.labelsize': 12
})
fig, (ax01, ax02, ax11, ax12) = plt.subplots(1, 4, figsize=(15,5), sharex=True)

c_eff_numerator_s01_biased1 = TTbar_biased_700to1000_unweighted['c_eff_numerator_s01'+binwidth].integrate('dataset', 'TTbar_biased_700to1000')
c_eff_numerator_s01_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_numerator_s01'+binwidth].integrate('dataset', 'TTbar_biased_1000toInf')
c_eff_denominator_s01_biased1 = TTbar_biased_700to1000_unweighted['c_eff_denominator_s01'+binwidth].integrate('dataset', 'TTbar_biased_700to1000')
c_eff_denominator_s01_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_denominator_s01'+binwidth].integrate('dataset', 'TTbar_biased_1000toInf')

c_eff_numerator_s01 = c_eff_numerator_s01_biased1.copy()
c_eff_numerator_s01.add(c_eff_numerator_s01_biased2)
c_eff_denominator_s01 = c_eff_denominator_s01_biased1.copy()
c_eff_denominator_s01.add(c_eff_denominator_s01_biased2)

c_eff_s01 = plotratio2d(c_eff_numerator_s01, c_eff_denominator_s01, ax=ax01, cbar=False, cmap='Greens')

ax01.set_title(r'c-tag $\epsilon_{01}\left(p_T,\ \eta\right)$')

#----------------------------------------------------------------------------------------#

c_eff_numerator_s02_biased1 = TTbar_biased_700to1000_unweighted['c_eff_numerator_s02'+binwidth].integrate('dataset', 'TTbar_biased_700to1000')
c_eff_numerator_s02_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_numerator_s02'+binwidth].integrate('dataset', 'TTbar_biased_1000toInf')
c_eff_denominator_s02_biased1 = TTbar_biased_700to1000_unweighted['c_eff_denominator_s02'+binwidth].integrate('dataset', 'TTbar_biased_700to1000')
c_eff_denominator_s02_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_denominator_s02'+binwidth].integrate('dataset', 'TTbar_biased_1000toInf')

c_eff_numerator_s02 = c_eff_numerator_s02_biased1.copy()
c_eff_numerator_s02.add(c_eff_numerator_s02_biased2)
c_eff_denominator_s02 = c_eff_denominator_s02_biased1.copy()
c_eff_denominator_s02.add(c_eff_denominator_s02_biased2)

c_eff_s02 = plotratio2d(c_eff_numerator_s02, c_eff_denominator_s02, ax=ax02, cbar=False, cmap='Greens')

ax02.set_title(r'c-tag $\epsilon_{02}\left(p_T,\ \eta\right)$')
ax02.set_ylabel('')
ax02.set_yticks([])

# #----------------------------------------------------------------------------------------#

c_eff_numerator_s11_biased1 = TTbar_biased_700to1000_unweighted['c_eff_numerator_s11'+binwidth].integrate('dataset', 'TTbar_biased_700to1000')
c_eff_numerator_s11_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_numerator_s11'+binwidth].integrate('dataset', 'TTbar_biased_1000toInf')
c_eff_denominator_s11_biased1 = TTbar_biased_700to1000_unweighted['c_eff_denominator_s11'+binwidth].integrate('dataset', 'TTbar_biased_700to1000')
c_eff_denominator_s11_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_denominator_s11'+binwidth].integrate('dataset', 'TTbar_biased_1000toInf')

c_eff_numerator_s11 = c_eff_numerator_s11_biased1.copy()
c_eff_numerator_s11.add(c_eff_numerator_s11_biased2)
c_eff_denominator_s11 = c_eff_denominator_s11_biased1.copy()
c_eff_denominator_s11.add(c_eff_denominator_s11_biased2)

c_eff_s11 = plotratio2d(c_eff_numerator_s11, c_eff_denominator_s11, ax=ax11, cbar=False, cmap='Greens')

ax11.set_title(r'c-tag $\epsilon_{11}\left(p_T,\ \eta\right)$')
ax11.set_ylabel('')
ax11.set_yticks([])

# #----------------------------------------------------------------------------------------#

c_eff_numerator_s12_biased1 = TTbar_biased_700to1000_unweighted['c_eff_numerator_s12'+binwidth].integrate('dataset', 'TTbar_biased_700to1000')
c_eff_numerator_s12_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_numerator_s12'+binwidth].integrate('dataset', 'TTbar_biased_1000toInf')
c_eff_denominator_s12_biased1 = TTbar_biased_700to1000_unweighted['c_eff_denominator_s12'+binwidth].integrate('dataset', 'TTbar_biased_700to1000')
c_eff_denominator_s12_biased2 = TTbar_biased_1000toInf_unweighted['c_eff_denominator_s12'+binwidth].integrate('dataset', 'TTbar_biased_1000toInf')

c_eff_numerator_s12 = c_eff_numerator_s12_biased1.copy()
c_eff_numerator_s12.add(c_eff_numerator_s12_biased2)
c_eff_denominator_s12 = c_eff_denominator_s12_biased1.copy()
c_eff_denominator_s12.add(c_eff_denominator_s12_biased2)

c_eff_s12 = plotratio2d(c_eff_numerator_s12, c_eff_denominator_s12, ax=ax12, cmap='Greens')#, cmap='plasma')

ax12.set_title(r'c-tag $\epsilon_{12}\left(p_T,\ \eta\right)$')
ax12.set_ylabel('')
ax12.set_yticks([])
ax12.set_xlim(0.,2000.)

# #----------------------------------------------------------------------------------------#
Dataset = 'TTbarBiasedSum'
fig.suptitle(Dataset + ' Sample')

filename = Dataset + '_ctag_efficiency' + filename_bw + '.png'

In [ ]:
SaveDirectory = maindirectory + '/FlavorTagEfficiencies/images/2D/ctags/' # split histograms into subdirectories
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
if PlotWithLargeBins == True:
    binwidth = '_manualbins'
    filename_bw = '_large_bins'
else:
    binwidth = ''
    filename_bw = ''
for dataset,output in Datasets.items():
    plt.rcParams.update({
    'font.size': 14,
    'axes.titlesize': 18,
    'axes.labelsize': 18,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12
    })
    fig, (ax01, ax02, ax11, ax12) = plt.subplots(1, 4, figsize=(15,5), sharex=True)

    c_eff_numerator_s01 = output['c_eff_numerator_s01'+binwidth].integrate('dataset', dataset)
    c_eff_denominator_s01 = output['c_eff_denominator_s01'+binwidth].integrate('dataset', dataset)

    c_eff_s01 = plotratio2d(c_eff_numerator_s01, c_eff_denominator_s01, ax=ax01, cbar=False, cmap='Greens')

    ax01.set_title(r'c-tag $\epsilon_{01}\left(p_T,\ \eta\right)$')
    if binwidth == '_manualbins':
        ax01.set_yticks([-2.4, -1.8, -1.2, -0.6, 0, 0.6, 1.2, 1.8, 2.4])
        ax01.set_yticklabels(['-2.4', '-1.8', '-1.2', '-0.6', '0', '0.6', '1.2', '1.8', '2.4'])
        ax01.set_xticks([0, 200, 400, 800, 1600, 3200])
        ax01.set_xticklabels(['', '.2', '.4', '.8', '1.6', '3.2'])
        ax01.set_xlabel(r'$Subjet\ p_T\ [TeV]$')
        ax01.grid()

    #----------------------------------------------------------------------------------------#

    c_eff_numerator_s02 = output['c_eff_numerator_s02'+binwidth].integrate('dataset', dataset)
    c_eff_denominator_s02 = output['c_eff_denominator_s02'+binwidth].integrate('dataset', dataset)

    c_eff_s02 = plotratio2d(c_eff_numerator_s02, c_eff_denominator_s02, ax=ax02, cbar=False, cmap='Greens')

    ax02.set_title(r'c-tag $\epsilon_{02}\left(p_T,\ \eta\right)$')
    ax02.set_ylabel('')
    if binwidth == '_manualbins':
        ax02.set_yticks([-2.4, -1.8, -1.2, -0.6, 0, 0.6, 1.2, 1.8, 2.4])
        ax02.set_yticklabels(['-2.4', '-1.8', '-1.2', '-0.6', '0', '0.6', '1.2', '1.8', '2.4'])
        ax02.set_xticks([0, 200, 400, 800, 1600, 3200])
        ax02.set_xticklabels(['', '.2', '.4', '.8', '1.6', '3.2'])
        ax02.set_xlabel(r'$Subjet\ p_T\ [TeV]$')
        ax02.grid()

    #----------------------------------------------------------------------------------------#

    c_eff_numerator_s11 = output['c_eff_numerator_s11'+binwidth].integrate('dataset', dataset)
    c_eff_denominator_s11 = output['c_eff_denominator_s11'+binwidth].integrate('dataset', dataset)

    c_eff_s11 = plotratio2d(c_eff_numerator_s11, c_eff_denominator_s11, ax=ax11, cbar=False, cmap='Greens')

    ax11.set_title(r'c-tag $\epsilon_{11}\left(p_T,\ \eta\right)$')
    ax11.set_ylabel('')
    if binwidth == '_manualbins':
        ax11.set_yticks([-2.4, -1.8, -1.2, -0.6, 0, 0.6, 1.2, 1.8, 2.4])
        ax11.set_yticklabels(['-2.4', '-1.8', '-1.2', '-0.6', '0', '0.6', '1.2', '1.8', '2.4'])
        ax11.set_xticks([0, 200, 400, 800, 1600, 3200])
        ax11.set_xticklabels(['', '.2', '.4', '.8', '1.6', '3.2'])
        ax11.set_xlabel(r'$Subjet\ p_T\ [TeV]$')
        ax11.grid()

    #----------------------------------------------------------------------------------------#

    c_eff_numerator_s12 = output['c_eff_numerator_s12'+binwidth].integrate('dataset', dataset)
    c_eff_denominator_s12 = output['c_eff_denominator_s12'+binwidth].integrate('dataset', dataset)

    c_eff_s12 = plotratio2d(c_eff_numerator_s12, c_eff_denominator_s12, ax=ax12, cmap='Greens')#, cmap='plasma')

    ax12.set_title(r'c-tag $\epsilon_{12}\left(p_T,\ \eta\right)$')
    ax12.set_ylabel('')
    if binwidth == '_manualbins':
        ax12.set_yticks([-2.4, -1.8, -1.2, -0.6, 0, 0.6, 1.2, 1.8, 2.4])
        ax12.set_yticklabels(['-2.4', '-1.8', '-1.2', '-0.6', '0', '0.6', '1.2', '1.8', '2.4'])
        ax12.set_xticks([0, 200, 400, 800, 1600, 3200])
        ax12.set_xticklabels(['', '.2', '.4', '.8', '1.6', '3.2'])
        ax12.set_xlabel(r'$Subjet\ p_T\ [TeV]$')
        ax12.grid()

    #----------------------------------------------------------------------------------------#
    fig.suptitle(dataset + ' Sample')

    filename = dataset + '_ctag_efficiency' + filename_bw + '.png'
    print(filename + ' generated')
    plt.savefig(SaveDirectory+filename, bbox_inches="tight")

# Plot udsg-tag $\epsilon\left(p_T,\ \eta\right)$
# Can either plot with small bins $(50\times50)$ or large bins $(10\times10)$

In [ ]:
SaveDirectory = maindirectory + '/FlavorTagEfficiencies/images/2D/udsgtags/' # split histograms into subdirectories
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
if PlotWithLargeBins == True:
    binwidth = '_manualbins'
    filename_bw = '_large_bins'
else:
    binwidth = ''
    filename_bw = ''

plt.rcParams.update({
'font.size': 14,
'axes.titlesize': 18,
'axes.labelsize': 18,
'xtick.labelsize': 12,
'ytick.labelsize': 12
})
fig, (ax01, ax02, ax11, ax12) = plt.subplots(1, 4, figsize=(15,5), sharex=True)

normmax = .2

udsg_eff_numerator_s01_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_numerator_s01'+binwidth].integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_numerator_s01_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_numerator_s01'+binwidth].integrate('dataset', 'TTbar_biased_1000toInf')
udsg_eff_denominator_s01_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_denominator_s01'+binwidth].integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_denominator_s01_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_denominator_s01'+binwidth].integrate('dataset', 'TTbar_biased_1000toInf')

udsg_eff_numerator_s01 = udsg_eff_numerator_s01_biased1.copy()
udsg_eff_numerator_s01.add(udsg_eff_numerator_s01_biased2)
udsg_eff_denominator_s01 = udsg_eff_denominator_s01_biased1.copy()
udsg_eff_denominator_s01.add(udsg_eff_denominator_s01_biased2)

udsg_eff_s01 = plotratio2d(udsg_eff_numerator_s01, udsg_eff_denominator_s01, ax=ax01, norm_max=normmax, cbar=False, cmap='Reds')

ax01.set_title(r'udsg-tag $\epsilon_{01}\left(p_T,\ \eta\right)$')

#----------------------------------------------------------------------------------------#

udsg_eff_numerator_s02_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_numerator_s02'+binwidth].integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_numerator_s02_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_numerator_s02'+binwidth].integrate('dataset', 'TTbar_biased_1000toInf')
udsg_eff_denominator_s02_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_denominator_s02'+binwidth].integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_denominator_s02_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_denominator_s02'+binwidth].integrate('dataset', 'TTbar_biased_1000toInf')

udsg_eff_numerator_s02 = udsg_eff_numerator_s02_biased1.copy()
udsg_eff_numerator_s02.add(udsg_eff_numerator_s02_biased2)
udsg_eff_denominator_s02 = udsg_eff_denominator_s02_biased1.copy()
udsg_eff_denominator_s02.add(udsg_eff_denominator_s02_biased2)

udsg_eff_s02 = plotratio2d(udsg_eff_numerator_s02, udsg_eff_denominator_s02, ax=ax02, norm_max=normmax, cbar=False, cmap='Reds')

ax02.set_title(r'udsg-tag $\epsilon_{02}\left(p_T,\ \eta\right)$')
ax02.set_ylabel('')
ax02.set_yticks([])

# #----------------------------------------------------------------------------------------#

udsg_eff_numerator_s11_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_numerator_s11'+binwidth].integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_numerator_s11_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_numerator_s11'+binwidth].integrate('dataset', 'TTbar_biased_1000toInf')
udsg_eff_denominator_s11_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_denominator_s11'+binwidth].integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_denominator_s11_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_denominator_s11'+binwidth].integrate('dataset', 'TTbar_biased_1000toInf')

udsg_eff_numerator_s11 = udsg_eff_numerator_s11_biased1.copy()
udsg_eff_numerator_s11.add(udsg_eff_numerator_s11_biased2)
udsg_eff_denominator_s11 = udsg_eff_denominator_s11_biased1.copy()
udsg_eff_denominator_s11.add(udsg_eff_denominator_s11_biased2)

udsg_eff_s11 = plotratio2d(udsg_eff_numerator_s11, udsg_eff_denominator_s11, ax=ax11, norm_max=normmax, cbar=False, cmap='Reds')

ax11.set_title(r'udsg-tag $\epsilon_{11}\left(p_T,\ \eta\right)$')
ax11.set_ylabel('')
ax11.set_yticks([])

# #----------------------------------------------------------------------------------------#

udsg_eff_numerator_s12_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_numerator_s12'+binwidth].integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_numerator_s12_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_numerator_s12'+binwidth].integrate('dataset', 'TTbar_biased_1000toInf')
udsg_eff_denominator_s12_biased1 = TTbar_biased_700to1000_unweighted['udsg_eff_denominator_s12'+binwidth].integrate('dataset', 'TTbar_biased_700to1000')
udsg_eff_denominator_s12_biased2 = TTbar_biased_1000toInf_unweighted['udsg_eff_denominator_s12'+binwidth].integrate('dataset', 'TTbar_biased_1000toInf')

udsg_eff_numerator_s12 = udsg_eff_numerator_s12_biased1.copy()
udsg_eff_numerator_s12.add(udsg_eff_numerator_s12_biased2)
udsg_eff_denominator_s12 = udsg_eff_denominator_s12_biased1.copy()
udsg_eff_denominator_s12.add(udsg_eff_denominator_s12_biased2)

udsg_eff_s12 = plotratio2d(udsg_eff_numerator_s12, udsg_eff_denominator_s12, ax=ax12, norm_max=normmax, cmap='Reds')#, cmap='plasma')

ax12.set_title(r'udsg-tag $\epsilon_{12}\left(p_T,\ \eta\right)$')
ax12.set_ylabel('')
ax12.set_yticks([])
ax12.set_xlim(0.,2000.)

# #----------------------------------------------------------------------------------------#
Dataset = 'TTbarBiasedSum'
fig.suptitle(Dataset + ' Sample')

filename = Dataset + '_udsgtag_efficiency' + filename_bw + '.png'

In [ ]:
SaveDirectory = maindirectory + '/FlavorTagEfficiencies/images/2D/udsgtags/' # split histograms into subdirectories
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
if PlotWithLargeBins == True:
    binwidth = '_manualbins'
    filename_bw = '_large_bins'
else:
    binwidth = ''
    filename_bw = ''
for dataset,output in Datasets.items():
    plt.rcParams.update({
    'font.size': 14,
    'axes.titlesize': 18,
    'axes.labelsize': 18,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12
    })
    fig, (ax01, ax02, ax11, ax12) = plt.subplots(1, 4, figsize=(15,5), sharex=True)

    udsg_eff_numerator_s01 = output['udsg_eff_numerator_s01'+binwidth].integrate('dataset', dataset)
    udsg_eff_denominator_s01 = output['udsg_eff_denominator_s01'+binwidth].integrate('dataset', dataset)

    udsg_eff_s01 = plotratio2d(udsg_eff_numerator_s01, udsg_eff_denominator_s01, ax=ax01, cbar=False, cmap='Reds')

    ax01.set_title(r'udsg-tag $\epsilon_{01}\left(p_T,\ \eta\right)$')
    if binwidth == '_manualbins':
        ax01.set_yticks([-2.4, -1.8, -1.2, -0.6, 0, 0.6, 1.2, 1.8, 2.4])
        ax01.set_yticklabels(['-2.4', '-1.8', '-1.2', '-0.6', '0', '0.6', '1.2', '1.8', '2.4'])
        ax01.set_xticks([0, 200, 400, 800, 1600, 3200])
        ax01.set_xticklabels(['0', '.2', '.4', '.8', '1.6', '3.2'])
        ax01.set_xlabel(r'$Subjet\ p_T\ [TeV]$')
        ax01.grid()

    #----------------------------------------------------------------------------------------#

    udsg_eff_numerator_s02 = output['udsg_eff_numerator_s02'+binwidth].integrate('dataset', dataset)
    udsg_eff_denominator_s02 = output['udsg_eff_denominator_s02'+binwidth].integrate('dataset', dataset)

    udsg_eff_s02 = plotratio2d(udsg_eff_numerator_s02, udsg_eff_denominator_s02, ax=ax02, cbar=False, cmap='Reds')

    ax02.set_title(r'udsg-tag $\epsilon_{02}\left(p_T,\ \eta\right)$')
    ax02.set_ylabel('')
    if binwidth == '_manualbins':
        ax02.set_yticks([-2.4, -1.8, -1.2, -0.6, 0, 0.6, 1.2, 1.8, 2.4])
        ax02.set_yticklabels(['-2.4', '-1.8', '-1.2', '-0.6', '0', '0.6', '1.2', '1.8', '2.4'])
        ax02.set_xticks([0, 200, 400, 800, 1600, 3200])
        ax02.set_xticklabels(['0', '.2', '.4', '.8', '1.6', '3.2'])
        ax02.set_xlabel(r'$Subjet\ p_T\ [TeV]$')
        ax02.grid()
    
    
    #----------------------------------------------------------------------------------------#

    udsg_eff_numerator_s11 = output['udsg_eff_numerator_s11'+binwidth].integrate('dataset', dataset)
    udsg_eff_denominator_s11 = output['udsg_eff_denominator_s11'+binwidth].integrate('dataset', dataset)

    udsg_eff_s11 = plotratio2d(udsg_eff_numerator_s11, udsg_eff_denominator_s11, ax=ax11, cbar=False, cmap='Reds')

    ax11.set_title(r'udsg-tag $\epsilon_{11}\left(p_T,\ \eta\right)$')
    ax11.set_ylabel('')
    if binwidth == '_manualbins':
        ax11.set_yticks([-2.4, -1.8, -1.2, -0.6, 0, 0.6, 1.2, 1.8, 2.4])
        ax11.set_yticklabels(['-2.4', '-1.8', '-1.2', '-0.6', '0', '0.6', '1.2', '1.8', '2.4'])
        ax11.set_xticks([0, 200, 400, 800, 1600, 3200])
        ax11.set_xticklabels(['', '.2', '.4', '.8', '1.6', '3.2'])
        ax11.set_xlabel(r'$Subjet\ p_T\ [TeV]$')
        ax11.grid()
    

    #----------------------------------------------------------------------------------------#

    udsg_eff_numerator_s12 = output['udsg_eff_numerator_s12'+binwidth].integrate('dataset', dataset)
    udsg_eff_denominator_s12 = output['udsg_eff_denominator_s12'+binwidth].integrate('dataset', dataset)

    udsg_eff_s12 = plotratio2d(udsg_eff_numerator_s12, udsg_eff_denominator_s12, ax=ax12, cmap='Reds')#, cmap='plasma')

    ax12.set_title(r'udsg-tag $\epsilon_{12}\left(p_T,\ \eta\right)$')
    ax12.set_ylabel('')
    if binwidth == '_manualbins':
        ax12.set_yticks([-2.4, -1.8, -1.2, -0.6, 0, 0.6, 1.2, 1.8, 2.4])
        ax12.set_yticklabels(['-2.4', '-1.8', '-1.2', '-0.6', '0', '0.6', '1.2', '1.8', '2.4'])
        ax12.set_xticks([0, 200, 400, 800, 1600, 3200])
        ax12.set_xticklabels(['', '.2', '.4', '.8', '1.6', '3.2'])
        ax12.set_xlabel(r'$Subjet\ p_T\ [TeV]$')
        ax12.grid()
    
    
    
#     ax12.xaxis.grid()
#     ax12.yaxis.grid()
    #----------------------------------------------------------------------------------------#
    fig.suptitle(dataset + ' Sample')

    filename = dataset + '_udsgtag_efficiency' + filename_bw + '.png'
    print(filename + ' generated')
    plt.savefig(SaveDirectory+filename, bbox_inches="tight")